# **Analise de anúncios de emprego em Glassdoor com PostgreSQL**

# **Planejamento**

### Informações sobre o conjunto de dados

O conjunto de dados deste projeto contém 672 linhas e 15 colunas para as variáveis listadas abaixo. Para obter mais informações sobre os dados, consulte a fonte em [Kaggle](https://www.kaggle.com/datasets/rashikrahmanpritom/data-science-job-posting-on-glassdoor).

| Variável               | Descrição                                           |
|------------------------|-----------------------------------------------------|
| index                  | Índice numérico de identificação do registro        |
| Job Title              | Título da vaga de emprego                           |
| Salary Estimate        | Faixa salarial estimada para o cargo                |
| Job Description        | Descrição detalhada das responsabilidades do cargo  |
| Rating                 | Avaliação da empresa (escala 0-5)                   |
| Company Name           | Nome completo da empresa contratante               |
| Location               | Localização física do cargo (cidade/estado)         |
| Headquarters           | Localização da sede matriz da empresa               |
| Size                   | Número total de funcionários na empresa             |
| Founded                | Ano de fundação da empresa                          |
| Type of ownership      | Tipo de empresa (ex: sem fins lucrativos/pública/privada)     |
| Industry               | Setor econômico principal da empresa                |
| Sector                 | Subdivisão específica do setor de atuação           |
| Revenue                | Receita anual gerada pela empresa                   |
| Competitors            | Principais concorrentes da empresa no mercado       |

### Objetivo  

Analisar descrições de vagas para identificar padrões de habilidades demandadas. 

## Etapa 1. Importação

* Importar pacotes
* Carregar conjunto de dados

### 1. Importar pacotes

In [8]:
# ====================================================================================
# INSTALAÇÃO DE DEPENDÊNCIAS EXTERNAS 
# ====================================================================================
# !pip install ipython-sql          # Integração SQL no Jupyter
# !pip install psycopg2-binary      # Conector PostgreSQL
# !pip install sqlalchemy           # ORM para conexão com banco de dados
# !pip install python-dotenv        # Gerenciamento de variáveis de ambiente

In [9]:
# ====================================================================================
# PACOTES OPERACIONAIS PADRÃO
# ====================================================================================
import numpy as np                   # Manipulação numérica
import pandas as pd                  # Manipulação de dados tabulares
import os                            # Interação com sistema operacional
import warnings                      # Controle de avisos
warnings.filterwarnings('ignore', category=DeprecationWarning)    # Opcional: Suprime avisos não críticos


# ====================================================================================
# PACOTES DE VISUALIZAÇÃO
# ====================================================================================
import matplotlib.pyplot as plt      # Gráficos estáticos
import seaborn as sns                # Estilos e visualizações avançadas

In [10]:
# ====================================================================================
# CONFIGURAÇÕES DO PANDAS (Opcional)
# ====================================================================================
pd.set_option('display.max_columns', None)  # Mostra todas as colunas
pd.set_option('display.float_format', lambda x: '%.2f' % x)  # Formatação de decimais

In [11]:
# ====================================================================================
# PACOTES PARA BANCO DE DADOS E AMBIENTE
# ====================================================================================
from dotenv import load_dotenv        # Carrega variáveis do arquivo .env
from sqlalchemy import create_engine, text  # Cria conexão com banco de dados

# Carrega variáveis de ambiente do arquivo .env
load_dotenv()

# Configuração da conexão com PostgreSQL (valores lidos do .env)
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

# Cria engine de conexão segura
engine = create_engine(
    f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

In [12]:
try:
    with engine.connect() as connection:
        print("Conexão bem-sucedida!")
except Exception as e:
    print(f"Erro ao conectar: {e}")

Conexão bem-sucedida!


In [13]:
## Carregar a extensão no notebook

## Conectar ao Banco de Dados PostgreSQL

In [14]:
%load_ext sql

%sql postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}

### 2. Carregar conjunto de dados

Leia o conjunto de dados chamado **`Uncleaned_DS_jobs.csv`.** 

In [16]:
# Carregar o conjunto de dados em um dataframe
df0 = pd.read_csv("D:/#1 Ofice Work/#3 Projeto/#1 Projetos de limpeza de dados/Glassdoor/Para GitHub/Uncleaned_DS_jobs.csv")

# Exibir as primeiras linhas do dataframe
df0.head()

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Sr Data Scientist,$137K-$171K (Glassdoor est.),Description\n\nThe Senior Data Scientist is re...,3.10,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"
1,1,Data Scientist,$137K-$171K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nJoin ...",4.20,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
2,2,Data Scientist,$137K-$171K (Glassdoor est.),Overview\n\n\nAnalysis Group is one of the lar...,3.80,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1
3,3,Data Scientist,$137K-$171K (Glassdoor est.),JOB DESCRIPTION:\n\nDo you have a passion for ...,3.50,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,4,Data Scientist,$137K-$171K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.90,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


Criando a tabela `ds_jobs_raw`.

In [18]:
# Inserir os dados no PostgreSQL
df0.to_sql('ds_jobs_raw', engine, if_exists='replace', index=False)

672

Para garantir a segurança dos dados originais vou criar uma nova tabela para executar as alterações chamada `ds_jobs`.

In [20]:
# Criar Backup da tabela
with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS ds_jobs CASCADE;"))  
    conn.execute(text("CREATE TABLE ds_jobs AS SELECT * FROM ds_jobs_raw;"))
    conn.commit()
    
# Verificar se a tabela foi criada corretamente
query = "SELECT * FROM ds_jobs LIMIT 10;"
df = pd.read_sql(query, engine)

# Exibir os primeiros registros
display(df)

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Sr Data Scientist,$137K-$171K (Glassdoor est.),Description\n\nThe Senior Data Scientist is re...,3.10,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"
1,1,Data Scientist,$137K-$171K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nJoin ...",4.20,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
2,2,Data Scientist,$137K-$171K (Glassdoor est.),Overview\n\n\nAnalysis Group is one of the lar...,3.80,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1
3,3,Data Scientist,$137K-$171K (Glassdoor est.),JOB DESCRIPTION:\n\nDo you have a passion for ...,3.50,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,4,Data Scientist,$137K-$171K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.90,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"
5,5,Data Scientist,$137K-$171K (Glassdoor est.),About Us:\n\nHeadquartered in beautiful Santa ...,4.20,HG Insights\n4.2,"Santa Barbara, CA","Santa Barbara, CA",51 to 200 employees,2010,Company - Private,Computer Hardware & Software,Information Technology,Unknown / Non-Applicable,-1
6,6,Data Scientist / Machine Learning Expert,$137K-$171K (Glassdoor est.),Posting Title\nData Scientist / Machine Learni...,3.90,Novartis\n3.9,"Cambridge, MA","Basel, Switzerland",10000+ employees,1996,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$10+ billion (USD),-1
7,7,Data Scientist,$137K-$171K (Glassdoor est.),Introduction\n\nHave you always wanted to run ...,3.50,iRobot\n3.5,"Bedford, MA","Bedford, MA",1001 to 5000 employees,1990,Company - Public,Consumer Electronics & Appliances Stores,Retail,$1 to $2 billion (USD),-1
8,8,Staff Data Scientist - Analytics,$137K-$171K (Glassdoor est.),Intuit is seeking a Staff Data Scientist to co...,4.40,Intuit - Data\n4.4,"San Diego, CA","Mountain View, CA",5001 to 10000 employees,1983,Company - Public,Computer Hardware & Software,Information Technology,$2 to $5 billion (USD),"Square, PayPal, H&R Block"
9,9,Data Scientist,$137K-$171K (Glassdoor est.),Ready to write the best chapter of your career...,3.60,XSELL Technologies\n3.6,"Chicago, IL","Chicago, IL",51 to 200 employees,2014,Company - Private,Enterprise Software & Network Solutions,Information Technology,Unknown / Non-Applicable,-1


## Etapa 2. Visão geral do datset inicial

- Entendendo as variáveis
- Limpando o conjunto de dados (dados ausentes, redundantes, outliers)

### 1. Reunir informações sobre os dados

In [23]:
# Informações básicas
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              672 non-null    int64  
 1   Job Title          672 non-null    object 
 2   Salary Estimate    672 non-null    object 
 3   Job Description    672 non-null    object 
 4   Rating             672 non-null    float64
 5   Company Name       672 non-null    object 
 6   Location           672 non-null    object 
 7   Headquarters       672 non-null    object 
 8   Size               672 non-null    object 
 9   Founded            672 non-null    int64  
 10  Type of ownership  672 non-null    object 
 11  Industry           672 non-null    object 
 12  Sector             672 non-null    object 
 13  Revenue            672 non-null    object 
 14  Competitors        672 non-null    object 
dtypes: float64(1), int64(2), object(12)
memory usage: 78.9+ KB


In [24]:
# Estatísticas descritivas para dados numéricos
df0.describe()

,index,Rating,Founded
count,672.00,672.00,672.00
mean,335.50,3.52,1635.53
std,194.13,1.41,756.75
min,0.00,-1.00,-1.00
25%,167.75,3.30,1917.75
50%,335.50,3.80,1995.00
75%,503.25,4.30,2009.00
max,671.00,5.00,2019.00


In [25]:
# Estatísticas descritivas para dados categóricos
df0.describe(include=['object'])

,Job Title,Salary Estimate,Job Description,Company Name,Location,Headquarters,Size,Type of ownership,Industry,Sector,Revenue,Competitors
count,672,672,672,672,672,672,672,672,672,672,672,672
unique,172,30,489,432,207,229,9,13,58,23,14,108
top,Data Scientist,$79K-$131K (Glassdoor est.),Job Overview: The Data Scientist is a key memb...,Hatch Data Inc,"San Francisco, CA","New York, NY",51 to 200 employees,Company - Private,-1,Information Technology,Unknown / Non-Applicable,-1
freq,337,32,12,12,69,33,135,397,71,188,213,501


### 2. Renomear colunas

Renomeando as colunas conforme necessário, padronizando os nomes, corrigindo erros de ortografia e tornando-os mais concisos quando aplicável.

In [28]:
with engine.connect() as conn:
    # Renomear 'index' para 'id'
    conn.execute(text('ALTER TABLE ds_jobs RENAME COLUMN "index" TO id;'))
    
    # Renomear 'Job Title' para 'job_title'
    conn.execute(text('ALTER TABLE ds_jobs RENAME COLUMN "Job Title" TO job_title;'))
    
    # Renomear 'Salary Estimate' para 'salary_estimate'
    conn.execute(text('ALTER TABLE ds_jobs RENAME COLUMN "Salary Estimate" TO salary_estimate;'))
    
    # Renomear 'Job Description' para 'job_description'
    conn.execute(text('ALTER TABLE ds_jobs RENAME COLUMN "Job Description" TO job_description;'))
    
    # Renomear 'Rating' para 'rating'
    conn.execute(text('ALTER TABLE ds_jobs RENAME COLUMN "Rating" TO rating;'))
    
    # Renomear 'Company Name' para 'company'
    conn.execute(text('ALTER TABLE ds_jobs RENAME COLUMN "Company Name" TO company;'))
    
    # Renomear 'Location' para 'location'
    conn.execute(text('ALTER TABLE ds_jobs RENAME COLUMN "Location" TO location;'))
    
    # Renomear 'Headquarters' para 'headquarters'
    conn.execute(text('ALTER TABLE ds_jobs RENAME COLUMN "Headquarters" TO headquarters;'))
    
    # Renomear 'Size' para 'size'
    conn.execute(text('ALTER TABLE ds_jobs RENAME COLUMN "Size" TO size;'))
    
    # Renomear 'Founded' para 'founded'
    conn.execute(text('ALTER TABLE ds_jobs RENAME COLUMN "Founded" TO founded;'))
    
    # Renomear 'Type of ownership' para 'type_ownership'
    conn.execute(text('ALTER TABLE ds_jobs RENAME COLUMN "Type of ownership" TO type_ownership;'))
    
    # Renomear 'Industry' para 'industry'
    conn.execute(text('ALTER TABLE ds_jobs RENAME COLUMN "Industry" TO industry;'))
    
    # Renomear 'Sector' para 'sector'
    conn.execute(text('ALTER TABLE ds_jobs RENAME COLUMN "Sector" TO sector;'))
    
    # Renomear 'Revenue' para 'revenue'
    conn.execute(text('ALTER TABLE ds_jobs RENAME COLUMN "Revenue" TO revenue;'))
    
    # Renomear 'Competitors' para 'competitors'
    conn.execute(text('ALTER TABLE ds_jobs RENAME COLUMN "Competitors" TO competitors;'))
    
    # Confirmar todas as alterações
    conn.commit()

# Verificar se a tabela foi modificada corretamente
query = "SELECT * FROM ds_jobs LIMIT 10;"
df = pd.read_sql(query, engine)

In [29]:
# Confirmando a alteração das colunas
df.head(1)

,id,job_title,salary_estimate,job_description,rating,company,location,headquarters,size,founded,type_ownership,industry,sector,revenue,competitors
0,0,Sr Data Scientist,$137K-$171K (Glassdoor est.),Description\n\nThe Senior Data Scientist is re...,3.10,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"


**Explicação das mudanças:**  

As alterações garantem padronização em snake_case e lowercase, melhorando a legibilidade, evitando erros em consultas SQL (como necessidade de aspas) e seguindo boas práticas de bancos de dados. 

| Variável           | Descrição                                           |
|--------------------|-----------------------------------------------------|
| id                 | Índice numérico de identificação do registro        |
| job_title          | Título da vaga de emprego                           |
| salary_estimate    | Faixa salarial estimada para o cargo                |
| job_description    | Descrição detalhada das responsabilidades do cargo  |
| rating             | Avaliação da empresa (escala 0-5)                   |
| company            | Nome completo da empresa contratante               |
| location           | Localização física do cargo (cidade/estado)         |
| headquarters       | Localização da sede matriz da empresa               |
| size               | Número total de funcionários na empresa             |
| founded            | Ano de fundação da empresa                          |
| type_ownership     | Tipo de empresa (ex: sem fins lucrativos/pública/privada)     |
| industry           | Setor econômico principal da empresa                |
| sector             | Subdivisão específica do setor de atuação           |
| revenue            | Receita anual gerada pela empresa                   |
| competitors        | Principais concorrentes da empresa no mercado       |

## Etapa 3. Limpeza de dados
- Esse processo foi feito em PostgreSQL e registado aqui atráves do pacote `sqlalchemy`

### 1. Erro ortográficos e tabulações 

A coluna `company` apresenta caracteres indesejados (como quebras de linha `\n` e espaços múltiplos), misturando o nome da empresa com avaliações numéricas. Para padronizar o formato, substituirei todas as sequências de espaços, tabs ou quebras de linha por um único espaço, garantindo legibilidade e consistência nos dados.

In [35]:
# Checando as nomenclaturas
with engine.connect() as conn:
    query = text("SELECT DISTINCT company FROM ds_jobs;")
 	
    # Carregar em um DataFrame
    df = pd.read_sql(query, engine)
    # Exibir o resultado
    display(df)

,company
0,Mteq\n3.7
1,Southwest Research Institute\n3.9
2,KeHE Distributors\n2.5
3,Conflux Systems Inc.\n4.5
4,Caterpillar\n3.7
...,...
427,United BioSource\n2.3
428,Joby Aviation\n4.3
429,Trexquant Investment\n4.0
430,Child Care Aware of America\n2.8


In [36]:
# Resolvendo a tabulação
with engine.connect() as conn:
    # Usar r-string para evitar o problema com as expressões regulares
    conn.execute(text(r"UPDATE ds_jobs SET company = regexp_replace(company, '[\s\t\n]+', ' ', 'g');"))
    conn.commit()

In [37]:
# Exibir os primeiros registros
with engine.connect() as conn:
    query = text("SELECT DISTINCT company FROM ds_jobs;")
    
    # Carregar em um DataFrame
    df = pd.read_sql(query, engine)
    
# Exibir o DataFrame 
df.head(10)

,company
0,Leidos 3.5
1,"GrainBridge, LLC"
2,"Criterion Systems, Inc. 3.8"
3,Big Huge Games 4.9
4,TECHNOCRAFT Solutions 3.4
5,Gap Inc. 3.5
6,Cook Children's Health Care System 3.8
7,Via Transportation 3.7
8,ICW Group 3.3
9,ASRC Federal Holding Company 3.4


In [38]:
# Resolvendo os números no final da string
with engine.connect() as conn:
    # Usar r-string para evitar o problema com as expressões regulares
    conn.execute(text(r"UPDATE ds_jobs SET company = CASE WHEN company LIKE '%_._' THEN REPLACE(company, RIGHT(company, 4), '') ELSE company END;"))
    conn.commit()

In [39]:
# Exibir os primeiros registros
with engine.connect() as conn:
    query = text("SELECT DISTINCT company FROM ds_jobs;")
 	
    # Carregar em um DataFrame
    df = pd.read_sql(query, engine)
    
# Exibir o resultado
df.head(10)

,company
0,"GrainBridge, LLC"
1,hc1
2,"Homology Medicines, Inc."
3,"ISYS Technologies, Inc."
4,Compass Consulting Group
5,Comcast
6,Argo Group US
7,Falcon IT & Staffing Solutions
8,II-VI Incorporated
9,webfx.com


#### Ações tomadas:

A remoção de avaliações numéricas (ex.: \n3.7) evita conflitos em agregações por nome de empresa, como contagem de vagas por organização.

<hr style="border: none; border-top: 1px dashed #ccc;">

A coluna `location` apresenta inconsistências, como registros marcados apenas como 'Remote' ou 'United States', e casos em que o estado é descrito por extenso em vez de sua sigla. Essas variações dificultam a padronização dos dados para análises precisas.

In [43]:
# Visualizar mudança após todas as atualizações
with engine.connect() as conn:
    select_query = text("""
    SELECT DISTINCT RIGHT(location, 2), location
    FROM ds_jobs 
    WHERE RIGHT(location, 2) IN('ah', 'as', 'es', 'ey', 'ia', 'te')
    ORDER BY 1
     """)
    
    # Carregar em um DataFrame
    df = pd.read_sql(select_query, engine)
    # Exibir o resultado
    display(df)

,right,location
0,ah,Utah
1,as,Texas
2,es,United States
3,ey,New Jersey
4,ia,California
5,te,Remote


In [44]:
with engine.connect() as conn:
    query = text("""
    UPDATE ds_jobs
    SET location =
        CASE
        -- Mapear nomes completos de estados
        WHEN location = 'Utah' THEN 'UT'
        WHEN location = 'Texas' THEN 'TX'
        WHEN location = 'California' THEN 'CA'
        WHEN location = 'New Jersey' THEN 'NJ'
  
        -- Tratar casos específicos
        WHEN location = 'United States' THEN NULL
        
        ELSE location 
        END;
    """)
    conn.execute(query)
    conn.commit()


#### Ações tomadas:

- Modifiquei os nomes completos dos estados pelas suas siglas.
- Modifiquei o registro 'United States' para `NULL`.

### 2. Valores ausentes

Verificando se há valores ausentes nos dados.

In [47]:
with engine.connect() as conn:
    query = text("""
SELECT column_name, 
       COUNT(*) AS total_rows,
       COUNT(*) FILTER (WHERE column_value IS NULL) AS null_rows,
       ROUND(COUNT(*) FILTER (WHERE column_value IS NULL)::numeric / COUNT(*) * 100, 2) AS null_percentage
FROM (
    SELECT column_name
    FROM information_schema.columns
    WHERE table_schema = 'public'
      AND table_name   = 'ds_jobs'
) AS cols
CROSS JOIN LATERAL (
    SELECT cols.column_name::text AS column_value
    FROM public.ds_jobs
) AS data(column_value)
GROUP BY column_name
ORDER BY null_rows DESC;
    """)
df = pd.read_sql(query, engine)
display(df) 

,column_name,total_rows,null_rows,null_percentage
0,sector,672,0,0.00
1,type_ownership,672,0,0.00
2,job_description,672,0,0.00
3,founded,672,0,0.00
4,job_title,672,0,0.00
5,company,672,0,0.00
6,salary_estimate,672,0,0.00
7,industry,672,0,0.00
8,revenue,672,0,0.00
9,location,672,0,0.00


O conjunto de dados não apresenta valores ausentes, porém pode haver inconsistências no tratamento desses valores.

#### `sector`:

In [50]:
with engine.connect() as conn:
    query = text("""
SELECT sector, COUNT(id) 
FROM ds_jobs 
GROUP BY sector
ORDER BY COUNT(id) DESC;
    """)
df = pd.read_sql(query, engine)
display(df) 

,sector,count
0,Information Technology,188
1,Business Services,120
2,-1,71
3,Biotech & Pharmaceuticals,66
4,Aerospace & Defense,46
5,Finance,33
6,Insurance,32
7,Manufacturing,23
8,Health Care,21
9,Government,17


>`-1` é como eles estão representados.

#### `type_ownership`:

In [53]:
with engine.connect() as conn:
    query = text("""
SELECT type_ownership, COUNT(id) 
FROM ds_jobs 
GROUP BY type_ownership
ORDER BY COUNT(id) DESC;
    """)
df = pd.read_sql(query, engine)
display(df) 

,type_ownership,count
0,Company - Private,397
1,Company - Public,153
2,Nonprofit Organization,36
3,Subsidiary or Business Segment,28
4,-1,27
5,Government,10
6,Other Organization,5
7,Unknown,4
8,Private Practice / Firm,4
9,College / University,3


#### `founded`:

In [55]:
with engine.connect() as conn:
    query = text("""
SELECT founded, COUNT(id) 
FROM ds_jobs 
GROUP BY founded
ORDER BY COUNT(id) DESC;
    """)
df = pd.read_sql(query, engine)
display(df) 

,founded,count
0,-1,118
1,2012,34
2,2011,25
3,2015,22
4,2010,22
...,...,...
98,1957,1
99,1949,1
100,1972,1
101,1953,1


#### `type_ownership`:

In [57]:
with engine.connect() as conn:
    query = text("""
SELECT type_ownership, COUNT(id) 
FROM ds_jobs 
GROUP BY type_ownership
ORDER BY COUNT(id) DESC;
    """)
df = pd.read_sql(query, engine)
display(df) 

,type_ownership,count
0,Company - Private,397
1,Company - Public,153
2,Nonprofit Organization,36
3,Subsidiary or Business Segment,28
4,-1,27
5,Government,10
6,Other Organization,5
7,Unknown,4
8,Private Practice / Firm,4
9,College / University,3


#### `job_title`:

In [59]:
with engine.connect() as conn:
    query = text("""
SELECT job_title, COUNT(id) 
FROM ds_jobs 
GROUP BY job_title
ORDER BY COUNT(id) DESC;
    """)
df = pd.read_sql(query, engine)
display(df) 

,job_title,count
0,Data Scientist,337
1,Data Engineer,26
2,Senior Data Scientist,19
3,Machine Learning Engineer,16
4,Data Analyst,12
...,...,...
167,"Data Engineer, Enterprise Analytics",1
168,Business Data Analyst,1
169,Report Writer-Data Analyst,1
170,"Machine Learning Scientist - Bay Area, CA",1


#### `salary_estimate`:

In [61]:
with engine.connect() as conn:
    query = text("""
SELECT salary_estimate, COUNT(id) 
FROM ds_jobs 
GROUP BY salary_estimate
ORDER BY COUNT(id) DESC;
    """)
df = pd.read_sql(query, engine)
display(df) 

,salary_estimate,count
0,$75K-$131K (Glassdoor est.),32
1,$79K-$131K (Glassdoor est.),32
2,$99K-$132K (Glassdoor est.),32
3,$137K-$171K (Glassdoor est.),30
4,$90K-$109K (Glassdoor est.),30
5,$79K-$106K (Glassdoor est.),22
6,$56K-$97K (Glassdoor est.),22
7,$90K-$124K (Glassdoor est.),22
8,$124K-$198K (Glassdoor est.),21
9,$112K-$116K (Glassdoor est.),21


#### `rating`:

In [63]:
with engine.connect() as conn:
    query = text("""
SELECT rating, COUNT(id) 
FROM ds_jobs 
GROUP BY rating
ORDER BY COUNT(id) DESC;
    """)
df = pd.read_sql(query, engine)
display(df) 

,rating,count
0,3.50,58
1,-1.00,50
2,4.00,41
3,3.30,41
4,3.90,40
5,3.80,39
6,3.70,38
7,5.00,36
8,4.50,32
9,3.60,31


#### `location`:

In [65]:
with engine.connect() as conn:
    query = text("""
SELECT location, COUNT(id) 
FROM ds_jobs 
GROUP BY location
ORDER BY COUNT(id) DESC;
    """)
df = pd.read_sql(query, engine)
display(df) 

,location,count
0,"San Francisco, CA",69
1,"New York, NY",50
2,"Washington, DC",26
3,"Boston, MA",24
4,"Chicago, IL",22
...,...,...
202,"Blue Bell, PA",1
203,"Rancho Cucamonga, CA",1
204,"Pleasanton, CA",1
205,"Washington, VA",1


#### `company`:

In [67]:
with engine.connect() as conn:
    query = text("""
SELECT company, COUNT(id) 
FROM ds_jobs 
GROUP BY company
ORDER BY COUNT(id) DESC;
    """)
df = pd.read_sql(query, engine)
display(df) 

,company,count
0,Hatch Data Inc,12
1,Maxar Technologies,12
2,Tempus Labs,11
3,AstraZeneca,10
4,Klaviyo,8
...,...,...
427,Conagen,1
428,Net2Source Inc.,1
429,Concerto HealthAI,1
430,"New Iron Group, Inc.",1


#### `headquarters`:

In [69]:
with engine.connect() as conn:
    query = text("""
SELECT headquarters, COUNT(id) 
FROM ds_jobs 
GROUP BY headquarters
ORDER BY COUNT(id) DESC;
    """)
df = pd.read_sql(query, engine)
display(df) 

,headquarters,count
0,"New York, NY",33
1,-1,31
2,"San Francisco, CA",31
3,"Chicago, IL",23
4,"Boston, MA",19
...,...,...
224,"Rome, NY",1
225,"Orange, CA",1
226,"Chennai, India",1
227,"Tel Aviv-Yafo, Israel",1


#### `size`:

In [71]:
with engine.connect() as conn:
    query = text("""
SELECT size, COUNT(id) 
FROM ds_jobs 
GROUP BY size
ORDER BY COUNT(id) DESC;
    """)
df = pd.read_sql(query, engine)
display(df) 

,size,count
0,51 to 200 employees,135
1,1001 to 5000 employees,104
2,1 to 50 employees,86
3,201 to 500 employees,85
4,10000+ employees,80
5,501 to 1000 employees,77
6,5001 to 10000 employees,61
7,-1,27
8,Unknown,17


#### `industry`:

In [73]:
with engine.connect() as conn:
    query = text("""
SELECT industry, COUNT(id) 
FROM ds_jobs 
GROUP BY industry
ORDER BY COUNT(id) DESC;
    """)
df = pd.read_sql(query, engine)
display(df)

,industry,count
0,-1,71
1,Biotech & Pharmaceuticals,66
2,IT Services,61
3,Computer Hardware & Software,57
4,Aerospace & Defense,46
5,Enterprise Software & Network Solutions,43
6,Consulting,38
7,Staffing & Outsourcing,36
8,Insurance Carriers,28
9,Internet,27


#### `revenue`:

In [75]:
with engine.connect() as conn:
    query = text("""
SELECT revenue, COUNT(id) 
FROM ds_jobs 
GROUP BY revenue
ORDER BY COUNT(id) DESC;
    """)
df = pd.read_sql(query, engine)
display(df) 

,revenue,count
0,Unknown / Non-Applicable,213
1,$100 to $500 million (USD),94
2,$10+ billion (USD),63
3,$2 to $5 billion (USD),45
4,$10 to $25 million (USD),41
5,$25 to $50 million (USD),36
6,$1 to $2 billion (USD),36
7,$50 to $100 million (USD),31
8,$1 to $5 million (USD),31
9,-1,27


#### `competitors`:

In [77]:
with engine.connect() as conn:
    query = text("""
SELECT competitors, COUNT(id) 
FROM ds_jobs 
GROUP BY competitors
ORDER BY COUNT(id) DESC;
    """)
df = pd.read_sql(query, engine)
display(df) 

,competitors,count
0,-1,501
1,"Roche, GlaxoSmithKline, Novartis",10
2,"Los Alamos National Laboratory, Battelle, SRI ...",6
3,"Leidos, CACI International, Booz Allen Hamilton",6
4,"Nielsen, Zappi, SurveyMonkey",3
...,...,...
103,"Thermo Fisher Scientific, Enzymatics, Illumina",1
104,"Liberty Mutual Insurance, EMPLOYERS, Travelers",1
105,"Epic, CipherHealth",1
106,"Adecco, ManpowerGroup, Allegis Corporation",1


---

#### Resumo da análise de Valores Ausentes  

O dataset apresenta valores ausentes representados por marcadores como **`-1`** e **`Unknown`**. Esses valores substituem dados faltantes, o que pode indicar uma estratégia de pré-processamento para evitar campos nulos. Abaixo, os principais pontos identificados:

**Colunas muito afetadas**  
- **`competitors`**:  
  - **501 registros (74,6%)** marcados como **`-1`**, indicando falta de informação sobre concorrentes.  
> - Isso sugere que a maioria das empresas não divulgou ou não possui dados sobre concorrentes.
> - Com `74,6%` dos registros sem concorrentes listados, análises de mercado ou competitividade podem ser bastante limitadas.

<blank>

- **`founded`**:  
  - **118 registros (17,5%)** com **`-1`**, indicando anos de fundação desconhecidos.  

<blank>

- **`industry`**:  
  - **71 registros (10,6%)** marcados como **`-1`**, sem classificação setorial.  

**Colunas menos afetadas**  
- **`type_ownership`**:  
  - **27 registros (4,0%)** como **`-1`** e **4 registros (0,6%)** como **`Unknown`**.  

<blank>

- **`headquarters`**:  
  - **31 registros (4,6%)** como **`-1`**, sem localização da sede.  

<blank>

- **`size`**:  
  - **27 registros (4,0%)** como **`-1`** e **17 (2,5%)** como **`Unknown`**, totalizando **6,6%** sem dados de porte.  

<blank>

- **`rating`**:  
  - **50 registros (7,4%)** com **`-1.00`**, possivelmente indicando avaliações não disponíveis.  

In [81]:
# Uniformizar os valores ausentes
with engine.connect() as conn:
    query = text("""
    UPDATE ds_jobs
    SET type_ownership = NULLIF(NULLIF(type_ownership, '-1'), 'Unknown'),
        size = NULLIF(NULLIF(size, '-1'), 'Unknown'),
        industry = NULLIF(industry, '-1'),
        sector = NULLIF(sector, '-1'),
        headquarters = NULLIF(headquarters, '-1'),
        competitors = NULLIF(competitors, '-1'),
        revenue = NULLIF(revenue, '-1'),
        founded = NULLIF(founded, '-1'),
        rating = NULLIF(rating, '-1.00');
    """)
    conn.execute(query)
    conn.commit()  # Fazer commit das alterações

Este código substitui valores que representam dados ausentes ou desconhecidos ('-1' e 'Unknown') por um valor nulo (`NULL`).

### 3. Dividindo `salary_estimate` e transformando em `INTEGER`

A coluna `salary_estimate` apresenta problemas com formato textual, o que dificulta cálculos estatísticos e comparações diretas. Para resolver isso, irei limpar a coluna e extrair os valores numéricos das estimativas mínima e máxima, organizando-os em duas novas colunas: **`salary_estimate_min`** e **`salary_estimate_max`**. Esse processo estrutura os dados para análises exploratórias e modelagem futura.

In [84]:
# Visualizando
with engine.connect() as conn:
    select_query = text("SELECT salary_estimate FROM ds_jobs LIMIT 10")
    df = pd.read_sql(select_query, engine)
    print(df)

                salary_estimate
0   $90K-$109K (Glassdoor est.)
1  $122K-$146K (Glassdoor est.)
2  $110K-$163K (Glassdoor est.)
3   $69K-$116K (Glassdoor est.)
4   $69K-$116K (Glassdoor est.)
5   $95K-$119K (Glassdoor est.)
6  $212K-$331K (Glassdoor est.)
7  $212K-$331K (Glassdoor est.)
8  $138K-$158K (Glassdoor est.)
9   $87K-$141K (Glassdoor est.)


In [85]:
# Limpando a coluna salary_estimate
with engine.connect() as conn:
    query = text("""
    UPDATE ds_jobs
    SET salary_estimate = REPLACE(REPLACE(salary_estimate, '(Glassdoor est.)', ''), '(Employer est.)', '');      
    """)
    conn.execute(query)
    conn.commit()

#### Criando `salary_estimate_lower`:

In [87]:
# Nova coluna
with engine.connect() as conn:
    query = text("""
    ALTER TABLE ds_jobs
    ADD COLUMN salary_estimate_lower TEXT
    """)
    conn.execute(query)
    conn.commit()

In [88]:
# Passando os dados para salary_estimate_lower
with engine.connect() as conn:
    query = text("""
    UPDATE ds_jobs
    SET salary_estimate_lower =  LEFT(salary_estimate, 4) 
    """)
    conn.execute(query)
    conn.commit()

In [89]:
# Removendo os símbolos
with engine.connect() as conn:
    query = text(r"""
    UPDATE ds_jobs
    SET salary_estimate_lower = REGEXP_REPLACE(salary_estimate_lower, '[^0-9]', '', 'g');
    """)
    conn.execute(query)
    conn.commit()

In [90]:
# Alterando o tipo de dados 
with engine.connect() as conn:
    query = text("""
    ALTER TABLE ds_jobs
    ALTER COLUMN salary_estimate_lower TYPE INTEGER
    USING salary_estimate_lower::INTEGER
    """)
    conn.execute(query)
    conn.commit()

In [91]:
# Convertendo os valores
with engine.connect() as conn:
    query = text("""
    UPDATE ds_jobs
    SET salary_estimate_lower = salary_estimate_lower * 1000;
    """)
    conn.execute(query)
    conn.commit()

#### Criando `salary_estimate_higher`:

In [93]:
# Nova coluna
with engine.connect() as conn:
    query = text("""
    ALTER TABLE ds_jobs
    ADD COLUMN salary_estimate_higher TEXT
    """)
    conn.execute(query)
    conn.commit()

In [94]:
# Passando os dados para salary_estimate_higher
with engine.connect() as conn:
    query = text("""
    UPDATE ds_jobs
    SET salary_estimate_higher = RIGHT(salary_estimate, 5)
    """)
    conn.execute(query)
    conn.commit()

In [95]:
# Removendo espaços desnecessários
with engine.connect() as conn:
    query = text(r"""
    UPDATE ds_jobs
    SET salary_estimate_higher = REGEXP_REPLACE(salary_estimate_higher, '[^a-zA-Z0-9]', '', 'g');
    """)
    conn.execute(query)
    conn.commit()

In [96]:
# Convertendo os valores
with engine.connect() as conn:
    query = text("""
    UPDATE ds_jobs
      SET salary_estimate_higher = CASE 
      WHEN salary_estimate_higher LIKE '%K' THEN (REPLACE(salary_estimate_higher, 'K', '')::NUMERIC * 1000)::INTEGER::TEXT
      ELSE salary_estimate_higher 
    END;
    """)
    conn.execute(query)
    conn.commit()

In [97]:
# Alterando o tipo de coluna
with engine.connect() as conn:
    query = text(r"""
    ALTER TABLE ds_jobs
    ALTER COLUMN salary_estimate_higher TYPE INTEGER
    USING salary_estimate_higher::INTEGER
    """)
    conn.execute(query)
    conn.commit()

#### Ações tomadas: 

**1. Extração dos Valores Numéricos**  
- **Remoção de texto redundante**:  
  Foram eliminados os sufixos *"(Glassdoor est.)"* e *"(Employer est.)"* usando `REPLACE`, deixando apenas a faixa salarial (ex.: *"\$90K-\$109K"*).  

- **Isolamento dos valores mínimo e máximo**:  
  - **Valor Mínimo (`salary_estimate_lower`)**:  
    Extraiu-se os primeiros 4 caracteres da string (ex.: *"\$90K"*) e removeu-se caracteres não numéricos com regex (`[^0-9]`), resultando em *"90"*.  
    Multiplicou-se por **1000** para converter "K" (milhares) em valores inteiros (ex.: `90 * 1000 = 90000`).  

  - **Valor Máximo (`salary_estimate_higher`)**:  
    Extraiu-se os últimos 5 caracteres (ex.: *"\$109K"*) e aplicou-se a mesma regex, resultando em *"109"*.  
    Também multiplicou-se por **1000** (ex.: `109 * 1000 = 109000`).  


**2. Conversão para Tipos Numéricos**  
- As novas colunas (`salary_estimate_lower` e `salary_estimate_higher`) foram definidas como `INTEGER` usando `ALTER TABLE`, garantindo que operações matemáticas possam ser realizadas sem erros de tipo.  


 **Impacto nas Análises Futuras**  
- Agora é possível calcular métricas como a **média salarial** (`(lower + higher) / 2`) ou a **amplitude da faixa salarial** (`higher - lower`).  
- Consultas como *"vagas com salário mínimo acima de \$100.000"* tornam-se muito mais fácil.  
- A remoção de símbolos e textos garante que análises estatísticas não sejam prejudicados por formatos inconsistentes. 

In [100]:
# Visualizando
with engine.connect() as conn:
    select_query = text("SELECT salary_estimate, salary_estimate_lower, salary_estimate_higher FROM ds_jobs LIMIT 10")
    df = pd.read_sql(select_query, engine)
    display(df)

,salary_estimate,salary_estimate_lower,salary_estimate_higher
0,$90K-$109K,90000,109000
1,$80K-$132K,80000,132000
2,$90K-$109K,90000,109000
3,$90K-$109K,90000,109000
4,$122K-$146K,122000,146000
5,$92K-$155K,92000,155000
6,$75K-$131K,75000,131000
7,$79K-$147K,79000,147000
8,$79K-$131K,79000,131000
9,$99K-$132K,99000,132000


### 4. Verificar duplicatas

Verificando se há entradas duplicadas nos dados.

In [102]:
# Identificar linhas duplicadas
with engine.connect() as conn:
    select_query = text("""
    WITH duplicates AS (
        SELECT 
            ctid,  -- Identificador físico da linha (opcional)
            ROW_NUMBER() OVER (
                PARTITION BY 
                    job_title, salary_estimate, job_description, rating, 
                    company, location, headquarters, size, founded, 
                    type_ownership, industry, sector, revenue
                ORDER BY ctid
            ) AS row_num  
        FROM ds_jobs
    )
    SELECT * 
    FROM duplicates 
    WHERE row_num > 1;  -- Filtra apenas duplicatas
    """)
    
    # Executa a consulta e carrega em um DataFrame
    df_duplicates = pd.read_sql(select_query, conn)
    
# Exibir o resultado
print(f"Total de linhas duplicadas: {len(df_duplicates)}")
print(df_duplicates)

Total de linhas duplicadas: 13
       ctid  row_num
0    (2,26)        2
1    (0,13)        2
2    (2,25)        3
3    (5,10)        4
4    (5,11)        5
5    (5,12)        6
6     (3,4)        2
7    (3,22)        3
8     (9,3)        4
9   (11,14)        5
10  (11,15)        6
11   (18,5)        2
12  (25,10)        2


Deletando as duplicatas. Uso uma Common Table Expression (CTE) e a função `ROW_NUMBER()`. A consulta particiona os dados por todas as colunas relevantes, numera as linhas idênticas e então deleta todas, exceto a primeira (`row_num > 1`) de cada grupo.

In [104]:
with engine.connect() as conn:
    # Query DELETE com CTE incorporada
    delete_query = text("""
        WITH duplicates AS (
            SELECT 
                ctid,
                ROW_NUMBER() OVER (
                    PARTITION BY 
                        job_title, salary_estimate, job_description, rating, 
                        company, location, headquarters, size, founded, 
                        type_ownership, industry, sector, revenue, competitors
                    ORDER BY ctid
                ) AS row_num  
            FROM ds_jobs
        )
        DELETE FROM ds_jobs
        WHERE ctid IN (SELECT ctid
                         FROM duplicates
                        WHERE row_num > 1);
    """)
    
    # Executar o DELETE
    conn.execute(delete_query)
    conn.commit() 

    # 2. Verificar se há duplicatas restantes
    check_duplicates_query = text("""
        WITH duplicates AS (
            SELECT 
                ctid,
                ROW_NUMBER() OVER (
                    PARTITION BY 
                        job_title, salary_estimate, job_description, rating, 
                        company, location, headquarters, size, founded, 
                        type_ownership, industry, sector, revenue, competitors
                    ORDER BY ctid) AS row_num  
            FROM ds_jobs
        )
        SELECT * 
        FROM duplicates 
        WHERE row_num > 1;
    """)
    
    # Carregar o resultado em um DataFrame
    df_remaining_duplicates = pd.read_sql(check_duplicates_query, conn)

# Exibir o resultado
print("Duplicatas restantes:", len(df_remaining_duplicates))

Duplicatas restantes: 0


#### Ações tomadas:

Duplicatas foram identificadas com base em combinações únicas de título, descrição e empresa. Foi mantida apenas a primeira ocorrência.

### 5. Verificar outliers

Verificando se há valores discrepantes nos dados.

#### `founded`:

In [108]:
with engine.connect() as conn:
    # Query para selecionar apenas a coluna founded
    stats_query = text("""
  SELECT DISTINCT company, founded
  FROM ds_jobs 
  ORDER BY founded
  LIMIT 20
    """)
    
    # Carregar o resultado em um DataFrame
    df_stats_query = pd.read_sql(stats_query, conn)

# Exibir o resultado
print(df_stats_query)

                          company  founded
0                          Takeda     1781
1               State of Virginia     1788
2                 US Pharmacopeia     1820
3                             GSK     1830
4                          Pfizer     1849
5                BWX Technologies     1850
6                      MassMutual     1851
7              First Health Group     1853
8                       Travelers     1853
9                   Guardian Life     1860
10                       Swiss Re     1863
11              Reynolds American     1875
12  The Davey Tree Expert Company     1880
13   Ameritas Life Insurance Corp     1887
14      Underwriters Laboratories     1894
15                          Roche     1896
16              Burns & McDonnell     1898
17      Carolina Power & Light Co     1908
18                            IBM     1911
19                           Mars     1911


Parece haver companhias mais antigas que merecem ser investigadas:

In [110]:
with engine.connect() as conn:
    stats_query = text("""
        WITH stats AS (
    SELECT
        percentile_cont(0.25) WITHIN GROUP (ORDER BY founded) AS q1,
        percentile_cont(0.75) WITHIN GROUP (ORDER BY founded) AS q3
    FROM ds_jobs
),
iqr_calc AS (
    SELECT 
        q1,
        q3,
        (q3 - q1) AS iqr,
        (q1 - 1.5 * (q3 - q1)) AS lower_bound,
        (q3 + 1.5 * (q3 - q1)) AS upper_bound
    FROM stats
)
SELECT 
    d.founded,
    i.lower_bound
FROM ds_jobs d, iqr_calc i
WHERE d.founded < i.lower_bound 
   OR d.founded > i.upper_bound
ORDER BY d.founded;
    """)
    
    # Carregar o resultado em um DataFrame
    df_stats_query = pd.read_sql(stats_query, conn)

# Exibir o resultado
print("Total de outliers:", len(df_stats_query))
print(df_stats_query)

Total de outliers: 52
    founded  lower_bound
0      1781      1920.00
1      1781      1920.00
2      1788      1920.00
3      1820      1920.00
4      1830      1920.00
5      1830      1920.00
6      1849      1920.00
7      1850      1920.00
8      1850      1920.00
9      1851      1920.00
10     1851      1920.00
11     1851      1920.00
12     1851      1920.00
13     1851      1920.00
14     1853      1920.00
15     1853      1920.00
16     1853      1920.00
17     1860      1920.00
18     1863      1920.00
19     1875      1920.00
20     1875      1920.00
21     1880      1920.00
22     1880      1920.00
23     1887      1920.00
24     1887      1920.00
25     1894      1920.00
26     1896      1920.00
27     1898      1920.00
28     1908      1920.00
29     1908      1920.00
30     1911      1920.00
31     1911      1920.00
32     1911      1920.00
33     1911      1920.00
34     1911      1920.00
35     1912      1920.00
36     1912      1920.00
37     1913      1920.00
38 

#### Observações:

Após a análise dos dados, foram identificadas as seguintes observações:

- **BWX Technologies:**  
  O arquivo indica o ano de fundação como **1850**. Contudo, dados históricos sugerem que a empresa, especializada em engenharia nuclear, foi fundada na década de 1950 (geralmente referenciada como 1956).

- **State of Virginia:**  
  Embora o ano **1788** esteja listado, é importante notar que este item refere-se a uma entidade governamental e não a uma empresa. O ano indicado possivelmente se relaciona ao período em que o estado ratificou a Constituição dos Estados Unidos, e não à fundação de uma organização empresarial.

- **GSK (GlaxoSmithKline):**  
  O ano **1830** pode estar associado à origem de um dos predecessores do grupo atual. A GSK resultou de várias fusões, e as datas históricas de seus componentes variam. Assim, o ano apresentado não reflete necessariamente a fundação do grupo como ele é conhecido atualmente.

Para as demais entidades listadas, os anos informados coincidem, de modo geral, com as datas amplamente reconhecidas em suas histórias institucionais.

In [113]:
# Corrigindo BWX Technologies
with engine.connect() as conn:
    query = text("""
    UPDATE ds_jobs
    SET founded = '1956'
    WHERE company =  'BWX Technologies'
      AND founded = '1850';      
    """)
    conn.execute(query)
    conn.commit()

<hr style="border: none; border-top: 1px dashed #ccc;">

#### `salary_estimate`:

Esta consulta utiliza o método do Intervalo Interquartil (IQR) para identificar outliers na coluna `salary_estimate_lower` da tabela `ds_jobs`. Ela calcula os limites inferior e superior com base no IQR e retorna os registros de salários que estão fora desses limites.

In [116]:
with engine.connect() as conn:
    # Query para selecionar apenas a coluna salary_estimate_lower
    stats_query = text("""
        WITH stats AS (
    SELECT
        percentile_cont(0.25) WITHIN GROUP (ORDER BY salary_estimate_lower) AS q1,
        percentile_cont(0.75) WITHIN GROUP (ORDER BY salary_estimate_lower) AS q3
    FROM ds_jobs
),
iqr_calc AS (
    SELECT 
        q1,
        q3,
        (q3 - q1) AS iqr,
        (q1 - 1.5 * (q3 - q1)) AS lower_bound,
        (q3 + 1.5 * (q3 - q1)) AS upper_bound
    FROM stats
)
SELECT 
    d.salary_estimate_lower,
    i.lower_bound,
    i.upper_bound
FROM ds_jobs d, iqr_calc i
WHERE d.salary_estimate_lower < i.lower_bound 
   OR d.salary_estimate_lower > i.upper_bound
ORDER BY d.founded;
    """)
    
    # Carregar o resultado em um DataFrame
    df_stats_query = pd.read_sql(stats_query, conn)

# Exibir o resultado
print("Total de outliers:", len(df_stats_query))
display(df_stats_query)

Total de outliers: 21


,salary_estimate_lower,lower_bound,upper_bound
0,212000,14500.00,186500.00
1,212000,14500.00,186500.00
2,212000,14500.00,186500.00
3,212000,14500.00,186500.00
4,212000,14500.00,186500.00
5,212000,14500.00,186500.00
6,212000,14500.00,186500.00
7,212000,14500.00,186500.00
8,212000,14500.00,186500.00
9,212000,14500.00,186500.00


<hr style="border: none; border-top: 1px dashed #ccc;">

Esta consulta faz o mesmo método para `salary_estimate_higher`.

In [118]:
with engine.connect() as conn:
    # Query para selecionar apenas a coluna salary_estimate_higher
    stats_query = text("""
        WITH stats AS (
    SELECT
        percentile_cont(0.25) WITHIN GROUP (ORDER BY salary_estimate_higher) AS q1,
        percentile_cont(0.75) WITHIN GROUP (ORDER BY salary_estimate_higher) AS q3
    FROM ds_jobs
),
iqr_calc AS (
    SELECT 
        q1,
        q3,
        (q3 - q1) AS iqr,
        (q1 - 1.5 * (q3 - q1)) AS lower_bound,
        (q3 + 1.5 * (q3 - q1)) AS upper_bound
    FROM stats
)
SELECT 
    d.salary_estimate_higher,
    i.lower_bound,
    i.upper_bound
FROM ds_jobs d, iqr_calc i
WHERE d.salary_estimate_higher < i.lower_bound 
   OR d.salary_estimate_higher > i.upper_bound
ORDER BY d.founded;
    """)
    
    # Carregar o resultado em um DataFrame
    df_stats_query = pd.read_sql(stats_query, conn)

# Exibir o resultado
print("Total de outliers:", len(df_stats_query))
display(df_stats_query)

Total de outliers: 21


,salary_estimate_higher,lower_bound,upper_bound
0,331000,50000.00,234000.00
1,331000,50000.00,234000.00
2,331000,50000.00,234000.00
3,331000,50000.00,234000.00
4,331000,50000.00,234000.00
5,331000,50000.00,234000.00
6,331000,50000.00,234000.00
7,331000,50000.00,234000.00
8,331000,50000.00,234000.00
9,331000,50000.00,234000.00


<hr style="border: none; border-top: 1px dashed #ccc;">

In [120]:
with engine.connect() as conn:
    stats_query = text("""
  SELECT company, salary_estimate_lower, salary_estimate_higher 
  FROM ds_jobs
  WHERE salary_estimate_lower = '212000'
  ORDER BY salary_estimate_lower
    """)
    
    # Carregar o resultado em um DataFrame
    df_stats_query = pd.read_sql(stats_query, conn)

# Exibir o resultado
display(df_stats_query)

,company,salary_estimate_lower,salary_estimate_higher
0,Klaviyo,212000,331000
1,Comtech Global Inc,212000,331000
2,Monte Rosa Therapeutics,212000,331000
3,CompuForce,212000,331000
4,Alaka`ina Foundation Family of Companies,212000,331000
5,Hexagon US Federal,212000,331000
6,Allen Institute,212000,331000
7,Southwest Research Institute,212000,331000
8,1-800-Flowers,212000,331000
9,Aptive,212000,331000


#### Observações:

Na análise identifiquei registros de salários que se encontram acima do limite superior (Q3) definido na análise estatística. Os principais detalhes são:

  - **Estimativa Salarial Baixa:**  
    - Valor observado: **212.000**  
    - Limite superior (Q3): 186.500  

<blank>
    
  - **Estimativa Salarial Alta:**  
    - Valor observado: **331.000**  
    - Limite superior (Q3): 234.000  

<blank>
    
  Esses achados indicam que alguns anúncios de emprego apresentam salários acima do intervalo considerado normal, sugerindo que certas empresas podem estar oferecendo pacotes salariais mais competitivos ou diferenciados.

- **Empresas com salários acima do limite superior:** 

1-800-Flowers, 10x Genomics, Alaka`ina Foundation Family of Companies, Allen Institute, Aptive, AstraZeneca, Aveshka, Inc., Blue Horizon Tek Solutions, CompuForce, Comtech Global Inc, Creative Circle, Hexagon US Federal, Klaviyo, Maxar Technologies, Monte Rosa Therapeutics, Roche, Sharpedge Solutions Inc, Smith Hanley Associates, Southwest Research Institute

<hr style="border: none; border-top: 1px dashed #ccc;">

#### `rating`:

In [124]:
with engine.connect() as conn:
    stats_query = text("""
        WITH stats AS (
    SELECT
        percentile_cont(0.25) WITHIN GROUP (ORDER BY rating) AS q1,
        percentile_cont(0.75) WITHIN GROUP (ORDER BY rating) AS q3
    FROM ds_jobs
),
iqr_calc AS (
    SELECT 
        q1,
        q3,
        (q3 - q1) AS iqr,
        (q1 - 1.5 * (q3 - q1)) AS lower_bound,
        (q3 + 1.5 * (q3 - q1)) AS upper_bound
    FROM stats
)
SELECT 
    d.rating,
    i.lower_bound,
    upper_bound
FROM ds_jobs d, iqr_calc i
WHERE d.rating < i.lower_bound 
   OR d.rating > i.upper_bound
ORDER BY d.rating;
    """)
    
    # Carregar o resultado em um DataFrame
    df_stats_query = pd.read_sql(stats_query, conn)

# Exibir o resultado
print("Total de outliers:", len(df_stats_query))
display(df_stats_query)

Total de outliers: 2


,rating,lower_bound,upper_bound
0,2.00,2.15,5.75
1,2.10,2.15,5.75


#### Observações:

Foram identificados dois registros que se configuram como outliers:

- **Detalhes dos Outliers:**
  - Valores observados: **2.00** e **2.10**
  - Limite inferior (Q1): **2.15**

<blank>
    
Apesar de classificados como outliers, os valores de 2.00 e 2.10 não estão muito abaixo do limite inferior estabelecido. Isso sugere que tais registros podem refletir avaliações reais de empresas que, embora ligeiramente abaixo da média, não configuram necessariamente erros na coleta ou registro dos dados. Portanto, esses outliers podem representar variações legítimas na percepção dos usuários sobre as empresas.

---

# **Analise**

## Etapa 4. Exploração de dados

### 1. Feature engineering

Nesta parte do projeto, projetarei alguns novos recursos para serem usados para modelagem.

#### Flags de Habilidades:

Irei criar colunas binárias (flags) para habilidades chave extraídas da coluna `job_description`. O objetivo é otimizar a análise, facilitando a contagem de vagas por habilidade, a identificação de combinações e a comparação de salários. 

In [133]:
# Adicionado as colunas binárias
with engine.connect() as conn:
    query = text("""
        ALTER TABLE ds_jobs
        ADD COLUMN tem_sql int,
        ADD COLUMN tem_python int,
        ADD COLUMN tem_excel int,
        ADD COLUMN tem_power_bi int,
        ADD COLUMN tem_tableau int,
        ADD COLUMN tem_machine_learning int,
        ADD COLUMN tem_cloud int;
    """)
    conn.execute(query)
    conn.commit()

Usando `UPDATE` para preencher as colunas com os resultados da consulta:

In [135]:
# Inserindo os dados nas colunas
with engine.connect() as conn:
    query = text(r"""
UPDATE ds_jobs
SET
    tem_sql = CASE WHEN LOWER(job_description) ~ '\msql\M' THEN 1 ELSE 0 END,
    tem_python = CASE WHEN LOWER(job_description) ~ '\mpython\M' THEN 1 ELSE 0 END,
    tem_excel = CASE WHEN LOWER(job_description) ~ '\mexcel\M' THEN 1 ELSE 0 END,
    tem_power_bi = CASE WHEN LOWER(job_description) ~ '\mpower\s*bi\M' THEN 1 ELSE 0 END,
    tem_tableau = CASE WHEN LOWER(job_description) ~ '\mtableau\M' THEN 1 ELSE 0 END,
    tem_machine_learning = CASE WHEN LOWER(job_description) ~ '\mmachine\s*learning\M|\mml\M' THEN 1 ELSE 0 END,
    tem_cloud = CASE WHEN LOWER(job_description) ~ '\mcloud\M|\maws\M|\mazure\M|\mgcp\M' THEN 1 ELSE 0 END;
    """)
    conn.execute(query)
    conn.commit()

# Visualizar mudança após todas as atualizações
with engine.connect() as conn:
    select_query = text("""
    SELECT id, job_title, tem_sql, tem_python, tem_excel, tem_power_bi, tem_tableau, tem_machine_learning, tem_cloud 
    FROM ds_jobs 
    ORDER BY id 
    LIMIT 10;
     """)
    
    # Carregar em um DataFrame
    df = pd.read_sql(select_query, engine)
    # Exibir o resultado
    display(df)

,id,job_title,tem_sql,tem_python,tem_excel,tem_power_bi,tem_tableau,tem_machine_learning,tem_cloud
0,0,Sr Data Scientist,0,0,0,0,0,1,1
1,1,Data Scientist,1,0,0,0,0,1,0
2,2,Data Scientist,0,1,0,0,0,1,1
3,3,Data Scientist,1,1,0,0,0,1,0
4,4,Data Scientist,1,1,0,0,0,1,1
5,5,Data Scientist,0,1,0,0,0,1,0
6,6,Data Scientist / Machine Learning Expert,0,1,0,0,0,1,0
7,7,Data Scientist,1,1,0,0,0,0,0
8,8,Staff Data Scientist - Analytics,0,0,0,0,0,0,0
9,9,Data Scientist,1,1,0,0,0,1,1


<hr style="border: none; border-top: 1px dashed #ccc;">

#### `salary_estimate_avg`:

Este código cria a coluna `salary_estimate_avg`, representando o salário médio. Ter um único valor numérico de salário facilita cálculos estatísticos, como média, mediana e desvio padrão, além de simplificar análises comparativas entre localização, setor, empresa e habilidades.

In [139]:
# Adicionado a coluna
with engine.connect() as conn:
    query = text("""
        ALTER TABLE ds_jobs
        ADD COLUMN salary_estimate_avg NUMERIC
    """)
    conn.execute(query)
    conn.commit()

In [140]:
# Inserindo os dados na coluna
with engine.connect() as conn:
    query = text("""
     UPDATE ds_jobs
     SET salary_estimate_avg = (salary_estimate_lower + salary_estimate_higher) / 2.0
                                                   -- 2.0 para garantir divisão float
    """)
    conn.execute(query)
    conn.commit()


# Visualizar mudança após todas as atualizações
with engine.connect() as conn:
    select_query = text("""
    SELECT id, salary_estimate, salary_estimate_lower, salary_estimate_higher, salary_estimate_avg
    FROM ds_jobs 
    ORDER BY id 
    LIMIT 10;
     """)
    
    # Carregar em um DataFrame
    df = pd.read_sql(select_query, engine)
    # Exibir o resultado
    display(df)

,id,salary_estimate,salary_estimate_lower,salary_estimate_higher,salary_estimate_avg
0,0,$137K-$171K,137000,171000,154000.00
1,1,$137K-$171K,137000,171000,154000.00
2,2,$137K-$171K,137000,171000,154000.00
3,3,$137K-$171K,137000,171000,154000.00
4,4,$137K-$171K,137000,171000,154000.00
5,5,$137K-$171K,137000,171000,154000.00
6,6,$137K-$171K,137000,171000,154000.00
7,7,$137K-$171K,137000,171000,154000.00
8,8,$137K-$171K,137000,171000,154000.00
9,9,$137K-$171K,137000,171000,154000.00


<hr style="border: none; border-top: 1px dashed #ccc;">

#### `state`:

A coluna `location` está no formato "Cidade, ESTADO". Irei extrair a sigla do estado, criando a coluna `state`, o que facilita a agregação e análise de salários por região.

In [144]:
# Adicionado a coluna
with engine.connect() as conn:
    query = text("""
        ALTER TABLE ds_jobs
        ADD COLUMN state VARCHAR(10)
    """)
    conn.execute(query)
    conn.commit()

In [145]:
# Inserindo os dados nas colunas
with engine.connect() as conn:
    query = text("""
        UPDATE ds_jobs
        SET state =
        CASE
        -- Tratar o formato padrão "Cidade, ST" primeiro
        WHEN location LIKE '%, %' THEN RIGHT(location, 2)

        -- Mapear os nomes separados
        WHEN LENGTH(location) = 2 THEN location
        
        -- Tratar caso específico
        WHEN location = 'Remote' THEN 'Remote'

        ELSE NULL 
        END;
    """)
    conn.execute(query)
    conn.commit()

# Visualizar mudança após todas as atualizações
with engine.connect() as conn:
    select_query = text("""
    SELECT id, location, state 
    FROM ds_jobs 
    ORDER BY id 
    LIMIT 10;
     """)
    
    # Carregar em um DataFrame
    df = pd.read_sql(select_query, engine)
    # Exibir o resultado
    display(df)

,id,location,state
0,0,"New York, NY",NY
1,1,"Chantilly, VA",VA
2,2,"Boston, MA",MA
3,3,"Newton, MA",MA
4,4,"New York, NY",NY
5,5,"Santa Barbara, CA",CA
6,6,"Cambridge, MA",MA
7,7,"Bedford, MA",MA
8,8,"San Diego, CA",CA
9,9,"Chicago, IL",IL


<hr style="border: none; border-top: 1px dashed #ccc;">

#### `job_title_simplified `:

Este código cria a coluna `job_title_simplified`, que agrupe títulos semelhantes. Por exemplo, "Senior Data Analyst", "Data Analyst II", "Lead Data Analyst" poderiam ser agrupados como "Data Analyst"; "Machine Learning Engineer", "AI Scientist" como "ML/AI Engineer/Scientist", etc. Isso reduzirá a granularidade e permitirá análises mais significativas por tipo de cargo, em vez de por cada variação específica do título, facilita a comparação de salários e habilidades requisitadas para funções equivalentes.

In [149]:
# Adicionado a coluna
with engine.connect() as conn:
    query = text("""
        ALTER TABLE ds_jobs
        ADD COLUMN job_title_simplified text
    """)
    conn.execute(query)
    conn.commit()

In [150]:
# Inserindo os dados nas colunas
with engine.connect() as conn:
    query = text("""
        UPDATE ds_jobs
        SET job_title_simplified = 
                              CASE 
                              WHEN job_title ILIKE '%Data Analyst%' THEN 'Data Analyst'
                              WHEN job_title ILIKE '%Machine Learning Engineer%' THEN 'ML/AI Engineer/Scientist'
                              WHEN job_title ILIKE '%Machine Learning%' THEN 'ML/AI Engineer/Scientist'
                              WHEN job_title ILIKE '%AI Scientist%' THEN 'ML/AI Engineer/Scientist'
                              WHEN job_title ILIKE '%Business Analyst%' THEN 'Business/BI Analyst'
                              WHEN job_title ILIKE '%Business%' THEN 'Business/BI Analyst'
                              WHEN job_title ILIKE '%Data Engineer%' THEN 'Data Engineer'
                              WHEN job_title ILIKE '%BI Analyst%' THEN 'Business/BI Analyst'
                              WHEN job_title ILIKE '%Statistician%' THEN 'Statistician'
                              WHEN job_title ILIKE '%Statistics%' THEN 'Statistician'
                              WHEN job_title ILIKE '%Research Scientist%' THEN 'Research Scientist'
                              WHEN job_title ILIKE '%Analytics Manager%' THEN 'Analytics Manager'
                              WHEN job_title ILIKE '%Data Scientist%' THEN 'Data Scientist'
                              WHEN job_title ILIKE '%Data Science%' THEN 'Data Scientist'
                              WHEN job_title ILIKE '%Scientist%' THEN 'Other Scientist'
                              ELSE 'Other' END;
    """)
    conn.execute(query)
    conn.commit()

# Visualizar mudança após todas as atualizações
with engine.connect() as conn:
    select_query = text("""
    SELECT job_title_simplified, COUNT(job_title_simplified) 
    FROM ds_jobs 
    GROUP BY job_title_simplified
    ORDER BY 2 DESC
     """)
    
    # Carregar em um DataFrame
    df = pd.read_sql(select_query, engine)
    # Exibir o resultado
    display(df)

,job_title_simplified,count
0,Data Scientist,443
1,ML/AI Engineer/Scientist,49
2,Data Analyst,47
3,Data Engineer,46
4,Other Scientist,43
5,Other,12
6,Business/BI Analyst,6
7,Statistician,6
8,Research Scientist,4
9,Analytics Manager,3


In [151]:
# Categoria Other
with engine.connect() as conn:
    select_query = text("""
SELECT job_title, COUNT(*)
FROM ds_jobs
WHERE job_title_simplified = 'Other'
GROUP BY job_title
ORDER BY COUNT(*) DESC
limit 50;
     """)
    
    # Carregar em um DataFrame
    df = pd.read_sql(select_query, engine)
    # Exibir o resultado
    display(df)

,job_title,count
0,Data Modeler (Analytical Systems),3
1,Data Analytics Engineer,2
2,Data Integration and Modeling Engineer,1
3,Data Modeler,1
4,Data Architect,1
5,Data Solutions Engineer - Data Modeler,1
6,Equity Data Insights Analyst - Quantitative An...,1
7,Principal Data & Analytics Platform Engineer,1
8,"Vice President, Biometrics and Clinical Data M...",1


##### Observações:

A categoria "Data Scientist" é a mais comum no dataset, com 443 registros, o que é esperado dado o foco do conjunto de dados "Data Science Job Posting on Glassdoor". Essa predominância reflete a natureza do dataset, mas não compromete a utilidade das demais categorias, que ainda representam 216 registros distribuídos entre funções como "Data Analyst" (47), "Data Engineer" (46) e "ML/AI Engineer/Scientist" (49).

Apesar da predominância de "Data Scientist", a coluna `job_title_simplified` mantém seu valor para análises específicas. Ela permite comparações entre diferentes funções — como salários médios, habilidades demandadas ou tendências regionais — sem a complexidade de lidar com múltiplas variações de títulos. A redução da categoria "Other" para 12 registros demonstra a eficácia do agrupamento, tornando as análises mais precisas e representativas das principais funções relacionadas a dados.

#### Adendo: Descarte da Feature `experience_level`

Inicialmente, considerei analisar o texto das descrições das vagas para extrair detalhes como nível de experiência exigido. No entanto, dada a complexidade da extração precisa dessa informação usando apenas SQL e o foco deste projeto em demonstrar análises com SQL padrão, optei por concentrar nas análises agregadas baseadas nas features estruturadas e nas flags de habilidades criadas, garantindo a viabilidade e a clareza do showcase.

### 2. Feature selection

Realizarei seleção de recursos para tornar o conjunto de dados mais enxuto, mantendo apenas as variáveis diretamente relevantes ou úteis para responder às perguntas principais. Isso resulta em um modelo mais focado, melhor interpretável e com maior eficiência.

Vou criar uma nova tabela para executar as alterações chamada `ds_jobs_fs`.

In [158]:
# Criar Backup da tabela
with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS ds_jobs_fs;"))    
    conn.execute(text("CREATE TABLE ds_jobs_fs AS SELECT * FROM ds_jobs;"))
    conn.commit()
    
# Verificar se a tabela foi criada corretamente
with engine.connect() as conn:
    query = "SELECT * FROM ds_jobs_fs LIMIT 3;"
    df = pd.read_sql(query, engine)

    # Exibir os primeiros registros
    display(df)

,id,job_title,salary_estimate,job_description,rating,company,location,headquarters,size,founded,type_ownership,industry,sector,revenue,competitors,salary_estimate_lower,salary_estimate_higher,tem_sql,tem_python,tem_excel,tem_power_bi,tem_tableau,tem_machine_learning,tem_cloud,salary_estimate_avg,state,job_title_simplified
0,154,ELISA RESEARCH SCIENTIST (CV-15),$90K-$109K,"Covaxx, a subsidiary of the UBI Group, has joi...",NaN,Covid-19 Search Partners,"Hauppauge, NY",None,None,NaN,None,None,None,None,None,90000,109000,0,0,0,0,0,0,0,99500.00,NY,Research Scientist
1,358,Data Scientist,$122K-$146K,Job Overview: The Data Scientist is a key memb...,NaN,Hatch Data Inc,"San Francisco, CA",None,None,NaN,None,None,None,None,None,122000,146000,0,1,0,0,0,1,1,134000.00,CA,Data Scientist
2,161,Say Business Data Analyst,$101K-$165K,A company built to serve you. It's your career...,4.10,Shelter Insurance,"Columbia, MO","Columbia, MO",1001 to 5000 employees,1946.00,Company - Private,Insurance Carriers,Insurance,$1 to $2 billion (USD),None,101000,165000,1,1,1,0,1,1,1,133000.00,MO,Data Analyst


<hr style="border: none; border-top: 1px dashed #ccc;">

Esta consulta SQL modifica a estrutura da tabela `ds_jobs_fs` removendo seis colunas específicas: `id`, `competitors`, `job_description`, `salary_estimate`, `founded` e `location`. O objetivo é limpar a tabela, eliminando dados considerados desnecessários ou irrelevantes para as análises subsequentes.

In [160]:
with engine.connect() as conn:
    query = text("""
        ALTER TABLE ds_jobs_fs
        DROP COLUMN id,
        DROP COLUMN competitors,
        DROP COLUMN job_description,
        DROP COLUMN salary_estimate,
        DROP COLUMN founded,
        DROP COLUMN location;
    """)
    conn.execute(query)
    conn.commit()

# Verificar se a tabela foi criada corretamente
with engine.connect() as conn:
    query = "SELECT * FROM ds_jobs_fs LIMIT 3;"
    df = pd.read_sql(query, engine)

    # Exibir os primeiros registros
    display(df)

,job_title,rating,company,headquarters,size,type_ownership,industry,sector,revenue,salary_estimate_lower,salary_estimate_higher,tem_sql,tem_python,tem_excel,tem_power_bi,tem_tableau,tem_machine_learning,tem_cloud,salary_estimate_avg,state,job_title_simplified
0,ELISA RESEARCH SCIENTIST (CV-15),NaN,Covid-19 Search Partners,None,None,None,None,None,None,90000,109000,0,0,0,0,0,0,0,99500.00,NY,Research Scientist
1,Data Scientist,NaN,Hatch Data Inc,None,None,None,None,None,None,122000,146000,0,1,0,0,0,1,1,134000.00,CA,Data Scientist
2,Say Business Data Analyst,4.10,Shelter Insurance,"Columbia, MO",1001 to 5000 employees,Company - Private,Insurance Carriers,Insurance,$1 to $2 billion (USD),101000,165000,1,1,1,0,1,1,1,133000.00,MO,Data Analyst


### 3. Análise de variáveis

#### Habilidades mais requisitadas:

Este código seleciona os dados de cada habilidade técnica (SQL, Python, Excel, etc.) na tabela `ds_jobs_fs`, somando as ocorrências de cada coluna 'tem_X' e combinando os resultados com UNION ALL, o que me permiti ordenar os totais de menções.

In [164]:
with engine.connect() as conn:
    analyze_query = text("""
        SELECT 
            'SQL' AS habilidade, SUM(tem_sql) AS total_mencoes FROM ds_jobs_fs UNION ALL
            SELECT 'Python', SUM(tem_python) FROM ds_jobs_fs UNION ALL
            SELECT 'Excel', SUM(tem_excel) FROM ds_jobs_fs UNION ALL
            SELECT 'Power BI', SUM(tem_power_bi) FROM ds_jobs_fs UNION ALL
            SELECT 'Tableau', SUM(tem_tableau) FROM ds_jobs_fs UNION ALL
            SELECT 'Machine Learning', SUM(tem_machine_learning) FROM ds_jobs_fs UNION ALL
            SELECT 'Cloud', SUM(tem_cloud) FROM ds_jobs_fs
        ORDER BY total_mencoes DESC;
    """)
 	
    # Carregar em um DataFrame
    df = pd.read_sql(analyze_query, engine)
    # Exibir o resultado
    display(df)

,habilidade,total_mencoes
0,Python,477
1,Machine Learning,409
2,SQL,323
3,Cloud,230
4,Tableau,122
5,Excel,78
6,Power BI,48


O código a seguir analisa as habilidades mais requisitadas em vagas de emprego, organizadas por `job_title_simplified`. Mostra a contagens absolutas das habilidades mais requisitadas.

In [166]:
with engine.connect() as conn:
    analyze_query = text("""
        SELECT 
            job_title_simplified,
            COUNT(*) AS numero_de_vagas,
            SUM(tem_sql) AS total_sql,
            SUM(tem_python) AS total_python,
            SUM(tem_excel) AS total_excel,
            SUM(tem_power_bi) AS total_power_bi,
            SUM(tem_tableau) AS total_tableau,
            SUM(tem_machine_learning) AS total_machine_learning,
            SUM(tem_cloud) AS total_cloud
        FROM ds_jobs_fs
        GROUP BY job_title_simplified
        ORDER BY job_title_simplified; 
    """)
 	
    # Carregar em um DataFrame
    df = pd.read_sql(analyze_query, engine)
    # Exibir o resultado
    display(df)

,job_title_simplified,numero_de_vagas,total_sql,total_python,total_excel,total_power_bi,total_tableau,total_machine_learning,total_cloud
0,Analytics Manager,3,3,0,0,0,3,0,0
1,Business/BI Analyst,6,5,2,2,4,2,1,0
2,Data Analyst,47,34,24,18,6,24,7,5
3,Data Engineer,46,30,37,3,6,6,14,32
4,Data Scientist,443,227,352,50,31,84,326,157
5,ML/AI Engineer/Scientist,49,13,40,1,0,0,49,24
6,Other,12,9,6,1,1,3,2,7
7,Other Scientist,43,2,15,0,0,0,7,5
8,Research Scientist,4,0,0,3,0,0,0,0
9,Statistician,6,0,1,0,0,0,3,0


Aqui para cada categoria, o código calcula a porcentagem de vagas que exigem habilidades específicas. Essas porcentagens são baseadas em colunas binárias (0 ou 1) que indicam se cada habilidade está presente na descrição da vaga.

In [168]:
with engine.connect() as conn:
    analyze_query = text("""
        SELECT 
            job_title_simplified,
            COUNT(*) AS numero_de_vagas,
            ROUND(SUM(tem_sql) * 100.0 / COUNT(*), 1) AS perc_sql,
            ROUND(SUM(tem_python) * 100.0 / COUNT(*), 1) AS perc_python,
            ROUND(SUM(tem_excel) * 100.0 / COUNT(*), 1) AS perc_excel,
            ROUND(SUM(tem_power_bi) * 100.0 / COUNT(*), 1) AS perc_power_bi,
            ROUND(SUM(tem_tableau) * 100.0 / COUNT(*), 1) AS perc_tableau,
            ROUND(SUM(tem_machine_learning) * 100.0 / COUNT(*), 1) AS perc_ml,
            ROUND(SUM(tem_cloud) * 100.0 / COUNT(*), 1) AS perc_cloud
        FROM ds_jobs_fs
        GROUP BY job_title_simplified
        ORDER BY job_title_simplified;
    """)
 	
    # Carregar em um DataFrame
    df = pd.read_sql(analyze_query, engine)
    # Exibir o resultado
    display(df)

,job_title_simplified,numero_de_vagas,perc_sql,perc_python,perc_excel,perc_power_bi,perc_tableau,perc_ml,perc_cloud
0,Analytics Manager,3,100.00,0.00,0.00,0.00,100.00,0.00,0.00
1,Business/BI Analyst,6,83.30,33.30,33.30,66.70,33.30,16.70,0.00
2,Data Analyst,47,72.30,51.10,38.30,12.80,51.10,14.90,10.60
3,Data Engineer,46,65.20,80.40,6.50,13.00,13.00,30.40,69.60
4,Data Scientist,443,51.20,79.50,11.30,7.00,19.00,73.60,35.40
5,ML/AI Engineer/Scientist,49,26.50,81.60,2.00,0.00,0.00,100.00,49.00
6,Other,12,75.00,50.00,8.30,8.30,25.00,16.70,58.30
7,Other Scientist,43,4.70,34.90,0.00,0.00,0.00,16.30,11.60
8,Research Scientist,4,0.00,0.00,75.00,0.00,0.00,0.00,0.00
9,Statistician,6,0.00,16.70,0.00,0.00,0.00,50.00,0.00


Agora vou agrupar as vagas por `setor` e calcular, para cada `setor`, o número total de vagas e a porcentagem de vagas que requerem habilidades específicas. Os resultados são ordenados de forma decrescente pelo número de vagas, facilitando a identificação dos setores com maior demanda.

In [170]:
with engine.connect() as conn:
    analyze_query = text("""
        SELECT 
            sector,
            COUNT(*) AS numero_de_vagas,
            ROUND(SUM(tem_sql) * 100.0 / COUNT(*), 1) AS perc_sql,
            ROUND(SUM(tem_python) * 100.0 / COUNT(*), 1) AS perc_python,
            ROUND(SUM(tem_excel) * 100.0 / COUNT(*), 1) AS perc_excel,
            ROUND(SUM(tem_power_bi) * 100.0 / COUNT(*), 1) AS perc_power_bi,
            ROUND(SUM(tem_tableau) * 100.0 / COUNT(*), 1) AS perc_tableau,
            ROUND(SUM(tem_machine_learning) * 100.0 / COUNT(*), 1) AS perc_ml,
            ROUND(SUM(tem_cloud) * 100.0 / COUNT(*), 1) AS perc_cloud
        FROM ds_jobs_fs
        WHERE sector IS NOT NULL
        GROUP BY sector
        ORDER BY numero_de_vagas DESC;
    """)
 	
    # Carregar em um DataFrame
    df = pd.read_sql(analyze_query, engine)
    # Exibir o resultado
    display(df)

,sector,numero_de_vagas,perc_sql,perc_python,perc_excel,perc_power_bi,perc_tableau,perc_ml,perc_cloud
0,Information Technology,187,44.40,80.70,9.60,2.70,13.90,70.60,44.40
1,Business Services,120,50.00,77.50,12.50,7.50,25.80,66.70,25.80
2,Biotech & Pharmaceuticals,66,40.90,62.10,13.60,3.00,7.60,59.10,39.40
3,Aerospace & Defense,46,34.80,69.60,6.50,8.70,10.90,63.00,19.60
4,Finance,32,56.30,62.50,31.30,15.60,40.60,37.50,34.40
5,Insurance,32,65.60,62.50,15.60,6.30,9.40,43.80,40.60
6,Manufacturing,23,65.20,65.20,8.70,30.40,26.10,47.80,34.80
7,Health Care,21,52.40,42.90,9.50,23.80,47.60,28.60,28.60
8,Government,17,29.40,88.20,0.00,0.00,41.20,70.60,23.50
9,"Oil, Gas, Energy & Utilities",10,50.00,20.00,20.00,0.00,0.00,70.00,20.00


#### Resumo:

A análise geral das habilidades mais mencionadas em todas as vagas revela um claro domínio de **Python (477 menções)** e **Machine Learning (409)**, seguidos de perto por **SQL (323)**. Habilidades de **Cloud (230)** e visualização como **Tableau (122)** também são significativas, enquanto ferramentas mais tradicionais como Excel (78) e Power BI (48) aparecem com menor frequência neste conjunto de dados.

Analisando as contagens absolutas por cargo: 
- **Data Scientists** (o grupo mais numeroso) demandam principalmente **Python (352), Machine Learning (326) e SQL (227)**.
- Para **Data Analysts**, as habilidades mais frequentes são **SQL (34), Python (24) e Tableau (24)**.
- **Data Engineers** mostram forte necessidade de **Python (37), Cloud (32) e SQL (30)**.
- E como esperado as vagas de **ML/AI Engineer/Scientist** focam intensamente em **Machine Learning (49), Python (40) e Cloud (24)**.

A análise percentual reforça a importância relativa das habilidades por cargo: 
- Destaca-se a exigência quase universal de **Machine Learning (100%)** para vagas de **ML/AI**, e de **SQL e Tableau (100% cada)** para **Analytics Manager**.
- Para **Data Analysts**, **SQL (72,3%)** é crucial, junto com **Python e Tableau (ambos 51,1%)**.
- **Data Engineers** precisam fortemente de **Python (80,4%) e Cloud (69,6%)**.
- Para **Data Scientists**, **Python (79,5%) e Machine Learning (73,6%)** são predominantes, com **SQL (51,2%)** sendo essencial para cerca de metade das vagas.

Observando a demanda por setor, nota-se variações significativas. 
- O setor de **Tecnologia da Informação** lidera em volume e exige fortemente **Python (80,7%) e Machine Learning (70,6%)**, além de Cloud e SQL.
- **Serviços de Negócios** também valorizam **Python (77,5%) e ML (66,7%)**, com maior ênfase em **Tableau (25,8%)** que o setor de TI.
- **Finanças** e **Seguros** mostram alta demanda por **SQL (56,3% e 65,6% respectivamente)** e **Python**, com Finanças utilizando mais **Tableau (40,6%)** e **Excel (31,3%)**.

É importante notar que setores com poucas vagas podem apresentar percentagens extremas (ex: 100%), que devem ser interpretadas com cautela devido ao baixo volume de dados.

<hr style="border: none; border-top: 1px dashed #ccc;">

#### Associação entre Habilidades e Salários:

Esta consulta SQL tem como objetivo comparar o impacto salarial do conjunto pré-definido de habilidades (SQL, Python, Excel, etc.). Ela cria uma lista dessas habilidades e, para cada uma, calcula a média e a mediana da estimativa salarial (`salary_estimate_avg`) separadamente para as vagas que *possuem* a habilidade (usando colunas booleanas como `tem_sql = 1`) e para as vagas que *não possuem* a habilidade (`tem_sql = 0`). A consulta utiliza `FILTER` dentro das funções de agregação (`AVG`, `PERCENTILE_CONT`) para realizar esses cálculos condicionais de forma eficiente. Por fim, os resultados são agrupados por habilidade e ordenados de forma decrescente pela mediana (e depois pela média) salarial das vagas *com* a habilidade.

In [175]:
with engine.connect() as conn:
    analyze_query = text("""
WITH skills_list (habilidade) AS (
    VALUES
        ('SQL'),
        ('Python'),
        ('Excel'),
        ('Power BI'),
        ('Tableau'),
        ('Machine Learning'),
        ('Cloud')
)
SELECT
    s.habilidade,
    -- Média salarial COM a habilidade
    AVG(j.salary_estimate_avg) FILTER (WHERE
        CASE s.habilidade
            WHEN 'SQL' THEN j.tem_sql = 1
            WHEN 'Python' THEN j.tem_python = 1
            WHEN 'Excel' THEN j.tem_excel = 1
            WHEN 'Power BI' THEN j.tem_power_bi = 1
            WHEN 'Tableau' THEN j.tem_tableau = 1
            WHEN 'Machine Learning' THEN j.tem_machine_learning = 1
            WHEN 'Cloud' THEN j.tem_cloud = 1
            ELSE FALSE
        END
    ) AS salario_medio_com_habilidade,
    -- Média salarial SEM a habilidade
    AVG(j.salary_estimate_avg) FILTER (WHERE
        CASE s.habilidade
            WHEN 'SQL' THEN j.tem_sql = 0
            WHEN 'Python' THEN j.tem_python = 0
            WHEN 'Excel' THEN j.tem_excel = 0
            WHEN 'Power BI' THEN j.tem_power_bi = 0
            WHEN 'Tableau' THEN j.tem_tableau = 0
            WHEN 'Machine Learning' THEN j.tem_machine_learning = 0
            WHEN 'Cloud' THEN j.tem_cloud = 0
            ELSE FALSE
        END
    ) AS salario_medio_sem_habilidade,
    -- Mediana salarial COM a habilidade
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY j.salary_estimate_avg) FILTER (WHERE
        CASE s.habilidade
            WHEN 'SQL' THEN j.tem_sql = 1
            WHEN 'Python' THEN j.tem_python = 1
            WHEN 'Excel' THEN j.tem_excel = 1
            WHEN 'Power BI' THEN j.tem_power_bi = 1
            WHEN 'Tableau' THEN j.tem_tableau = 1
            WHEN 'Machine Learning' THEN j.tem_machine_learning = 1
            WHEN 'Cloud' THEN j.tem_cloud = 1
            ELSE FALSE
        END
    ) AS salario_mediana_com_habilidade,
    -- Mediana salarial SEM a habilidade
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY j.salary_estimate_avg) FILTER (WHERE
        CASE s.habilidade
            WHEN 'SQL' THEN j.tem_sql = 0
            WHEN 'Python' THEN j.tem_python = 0
            WHEN 'Excel' THEN j.tem_excel = 0
            WHEN 'Power BI' THEN j.tem_power_bi = 0
            WHEN 'Tableau' THEN j.tem_tableau = 0
            WHEN 'Machine Learning' THEN j.tem_machine_learning = 0
            WHEN 'Cloud' THEN j.tem_cloud = 0
            ELSE FALSE
        END
    ) AS salario_mediana_sem_habilidade
FROM
    ds_jobs_fs j
CROSS JOIN
    skills_list s
GROUP BY
    s.habilidade
ORDER BY
    salario_mediana_com_habilidade DESC NULLS LAST,
    salario_medio_com_habilidade DESC NULLS LAST;
    """)
    
    # Carregar em um DataFrame
    df = pd.read_sql(analyze_query, engine)
    # Exibir o resultado
    display(df)

,habilidade,salario_medio_com_habilidade,salario_medio_sem_habilidade,salario_mediana_com_habilidade,salario_mediana_sem_habilidade
0,Power BI,125302.08,123644.84,120500.00,114000.00
1,Tableau,125692.62,123327.75,114000.00,114000.00
2,Machine Learning,125530.56,120878.00,114000.00,114000.00
3,SQL,124148.61,123397.32,114000.00,114000.00
4,Python,123561.84,124299.45,114000.00,114000.00
5,Cloud,121828.26,124804.20,114000.00,114000.00
6,Excel,119391.03,124352.84,113000.00,114000.00


#### Resumo:

A análise comparou os salários médios e medianos entre vagas que mencionam habilidades específicas versus as que não mencionam, dentro deste conjunto de dados. 
- Notavelmente, a presença da habilidade **Power BI** está associada a uma mediana salarial mais alta (120.5k contra 114k). 
- Em contraste, a menção de **Excel** correlaciona-se com uma mediana salarial ligeiramente inferior (113k contra 114k). 
- Para as demais habilidades técnicas principais analisadas (**Python, SQL, Machine Learning, Cloud, Tableau**), a mediana salarial mostrou-se idêntica (114k) independentemente da presença ou ausência da habilidade, apesar de pequenas variações observadas nas médias. 

É importante notar que estes achados refletem associações específicas deste dataset desbalanceado e não controlam outros fatores cruciais que influenciam a remuneração, como cargo, senioridade ou localização. Por exemplo, talvez **Excel** seja mencionado com mais frequência em vagas de nível júnior neste dataset, o que poderia explicar a média ligeiramente menor quando a habilidade está presente.

<hr style="border: none; border-top: 1px dashed #ccc;">

#### Cruzamento de Habilidades mais requisitadas vs. Associação entre Habilidades e Salários:

1.  **Power BI:**
    *   **Demanda:** É a habilidade com a *menor* demanda geral (48 menções). Sua demanda percentual é alta apenas em nichos como *Business/BI Analyst* (67%) e setores como *Manufacturing* (30%), *Health Care* (24%), *Finance* (16%).
    *   **Salário:** Apesar da baixa demanda geral, é a *única* habilidade onde a presença está associada a uma *mediana salarial notavelmente mais alta* (120.5k vs 114k).
    *   **Insight:** Isso sugere que, embora menos vagas exijam Power BI neste dataset, as que o fazem podem ser mais especializadas ou estar em setores/cargos onde o nível salarial base tende a ser um pouco maior *para aquele tipo de função*. A baixa frequência geral significa que esse resultado pode ser influenciado por um conjunto menor e potencialmente mais específico de vagas.
        
<blank>

2.  **Excel:**
    *   **Demanda:** Tem demanda geral baixa (78 menções), sendo mais relevante percentualmente para *Data Analysts* (38%) e *Business/BI Analysts* (33%), além do setor de *Finance* (31%).
    *   **Salário:** Sua presença está associada a uma *mediana salarial ligeiramente inferior* (113k vs 114k).
    *   **Insight:** Isso reforça a ideia de que Excel, embora necessário em certos papéis analíticos (especialmente os mais voltados a negócios/finanças), pode ser mais comum em vagas de nível de entrada ou menos técnicas *dentro deste pool de vagas de Data Science*, não sendo um fator que eleva a remuneração média/mediana no contexto geral de dados avançados.
        
<blank>

3.  **Python, ML, SQL: Essenciais:**
    *   **Demanda:** São as 3 habilidades *mais demandadas* no geral e cruciais para os cargos mais volumosos (*Data Scientist, Data Engineer, ML/AI Engineer*).
    *   **Salário:** Sua simples presença ou ausência *não altera a mediana salarial* (114k).
    *   **Insight:** Essas habilidades parecem ser o custo de entrada para a maioria dos papéis de dados neste nível. Sua alta presença significa que a variação salarial provavelmente depende muito mais de *outros fatores* (senioridade, combinações de outras skills, especialização em ML, localização, empresa) do que apenas ter ou não ter Python/SQL/ML listado.
        
<blank>

4.  **Tableau vs. Power BI:**
    *   **Demanda:** Tableau é consideravelmente *mais demandado* (122 menções) que Power BI (48), especialmente para *Data Analysts* (51%) e *Analytics Managers* (100%).
    *   **Salário:** A presença de Tableau *não* está associada a uma mudança na mediana salarial, ao contrário de Power BI.
    *   **Insight:** Embora mais popular e requisitado (especialmente para funções de análise), Tableau não mostra a mesma associação com salários medianos mais altos que Power BI neste dataset. Isso pode indicar que Tableau é usado em uma gama mais ampla de níveis de senioridade ou tipos de empresa, enquanto Power BI (neste dataset) pode estar concentrado em nichos específicos com remuneração ligeiramente superior, como discutido acima.
        
<blank>

5.  **Cloud:**
    *   **Demanda:** Tem demanda significativa (4º lugar, 230 menções), sendo vital para *Data Engineers* (70%) e *ML/AI Engineers* (49%).
    *   **Salário:** Não impacta a mediana salarial.
    *   **Insight:** Similar a Python/SQL/ML, as habilidades de Cloud são essenciais para funções específicas (principalmente engenharia), mas a complexidade (plataforma específica, nível de arquitetura) e outros fatores provavelmente ditam o salário mais do que a simples menção da necessidade de Cloud.
        
<blank>

**Conclusão:**

O cruzamento revela uma dinâmica interessante: 
- As habilidades mais presentes e fundamentais (Python, SQL, ML) não diferenciam a *mediana* salarial por si só, provavelmente porque são esperadas.
- Habilidades mais de nicho ou associadas a funções específicas mostram associações salariais distintas: Power BI (raro, mediana mais alta) e Excel (raro, mediana mais baixa).
- Ferramentas de visualização populares como Tableau, apesar de mais demandadas que Power BI, não mostram a mesma associação positiva com a mediana salarial neste conjunto de dados.

Isso sublinha a importância de olhar além da simples presença de uma habilidade e considerar o contexto do cargo, setor e a combinação de habilidades para entender a remuneração.

<hr style="border: none; border-top: 1px dashed #ccc;">

#### Variação de salários por estado e setor:

##### Faixa salarial por **cargo**:

A faixa salarial reflete os valores mínimo e máximo estimados, indicando a amplitude das ofertas no mercado para cada cargo em `job_title_simplified`. 

In [183]:
with engine.connect() as conn:
    analyze_query = text("""
        SELECT 
            job_title_simplified,
            MIN(salary_estimate_lower),
            MAX(salary_estimate_higher)
        FROM ds_jobs_fs
        GROUP BY job_title_simplified
        ORDER BY 1; 
    """)
 	
    # Carregar em um DataFrame
    df = pd.read_sql(analyze_query, engine)
    # Exibir o resultado
    display(df)

,job_title_simplified,min,max
0,Analytics Manager,79000,116000
1,Business/BI Analyst,31000,171000
2,Data Analyst,31000,225000
3,Data Engineer,31000,201000
4,Data Scientist,31000,331000
5,ML/AI Engineer/Scientist,31000,331000
6,Other,66000,201000
7,Other Scientist,31000,331000
8,Research Scientist,56000,123000
9,Statistician,56000,171000


A consulta mostra os desvios padrões dentro de cada categoria de `job_title_simplified`. Cargos com alto desvio padrão podem incluir uma ampla gama de níveis de experiência ou especializações.

In [185]:
with engine.connect() as conn:
    analyze_query = text("""
        SELECT 
            job_title_simplified,
            COUNT(job_title_simplified),
            STDDEV_SAMP(salary_estimate_avg) as salary_stddev
        FROM ds_jobs_fs
        GROUP BY job_title_simplified
        ORDER BY 3 DESC; 
    """)
 	
    # Carregar em um DataFrame
    df = pd.read_sql(analyze_query, engine)
    # Exibir o resultado
    display(df)

,job_title_simplified,count,salary_stddev
0,Other Scientist,43,52730.45
1,ML/AI Engineer/Scientist,49,41028.62
2,Data Scientist,443,40859.51
3,Business/BI Analyst,6,38127.96
4,Data Analyst,47,30876.46
5,Statistician,6,29204.02
6,Other,12,25651.62
7,Data Engineer,46,22743.54
8,Research Scientist,4,11145.81
9,Analytics Manager,3,10965.86


Aqui calculo a média e a mediana por categoria de `job_title_simplified`.

In [187]:
with engine.connect() as conn:
    analyze_query = text("""
        SELECT 
            job_title_simplified,
            COUNT(job_title_simplified) AS numero_de_vagas,
            AVG(salary_estimate_avg) AS average_salary,
            PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY salary_estimate_avg) AS median_salary
        FROM ds_jobs_fs
        GROUP BY job_title_simplified
        ORDER BY 2 DESC; 
    """)
 	
    # Carregar em um DataFrame
    df = pd.read_sql(analyze_query, engine)
    # Exibir o resultado
    display(df)

,job_title_simplified,numero_de_vagas,average_salary,median_salary
0,Data Scientist,443,125282.17,114000.00
1,ML/AI Engineer/Scientist,49,120969.39,107000.00
2,Data Analyst,47,117787.23,113000.00
3,Data Engineer,46,114119.57,113500.00
4,Other Scientist,43,138686.05,136500.00
5,Other,12,118875.00,113500.00
6,Statistician,6,106250.00,105000.00
7,Business/BI Analyst,6,107083.33,109250.00
8,Research Scientist,4,93125.00,98250.00
9,Analytics Manager,3,102000.00,99500.00


##### Faixa salarial por **estado**:

A consulta tem como objetivo extrair informações sobre salários estimados para diferentes estados, agrupando os resultados por `state` e ordenando-os alfabeticamente.

In [190]:
with engine.connect() as conn:
    analyze_query = text("""
        SELECT 
            state,
            MIN(salary_estimate_lower),
            MAX(salary_estimate_higher)
        FROM ds_jobs_fs
        GROUP BY state
        ORDER BY 3 DESC; 
    """)
 	
    # Carregar em um DataFrame
    df = pd.read_sql(analyze_query, engine)
    # Exibir o resultado
    display(df)

,state,min,max
0,TX,56000,331000
1,OH,31000,331000
2,CA,31000,331000
3,NY,31000,331000
4,MD,31000,331000
5,DE,212000,331000
6,None,31000,331000
7,DC,31000,331000
8,WA,56000,331000
9,MA,31000,331000


A consulta analisa a distribuição de vagas e a variação dos salários estimados médios por estado. Ela conta o número de vagas e calcula o desvio padrão dos salários para estados com mais de uma vaga. Os resultados são ordenados pelo desvio padrão em ordem decrescente, destacando os estados com maior disparidade salarial.

In [192]:
with engine.connect() as conn:
    analyze_query = text("""
        SELECT 
            state,
            COUNT(state) AS vagas_por_estado,
            STDDEV_SAMP(salary_estimate_avg) as salary_stddev
        FROM ds_jobs_fs
        WHERE state IS NOT NULL
        GROUP BY state
        HAVING COUNT(state) != 1
        ORDER BY 3 DESC; 
    """)
 	
    # Carregar em um DataFrame
    df = pd.read_sql(analyze_query, engine)
    # Exibir o resultado
    display(df)

,state,vagas_por_estado,salary_stddev
0,OK,6,77864.25
1,RI,2,76721.09
2,DC,26,62362.01
3,IA,3,56991.96
4,WA,16,55875.45
5,MI,5,55609.13
6,OH,14,52686.93
7,NC,9,49647.37
8,NY,52,47866.55
9,TX,18,47214.64


Esta consulta compila o número de vagas e, para cada estado com dados disponíveis, determina a média e a mediana dos salários estimados. A consulta filtra registros com estados nulos e os ordena em ordem decrescente de quantidade de vagas, destacando as regiões com maior oferta. 

Estados com poucas vagas podem ter estatísticas menos representativas devido ao tamanho da amostra.

In [194]:
with engine.connect() as conn:
    analyze_query = text("""
        SELECT 
            state,
            COUNT(state) AS vagas_por_estado,
            AVG(salary_estimate_avg) AS average_salary,
            PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY salary_estimate_avg) AS median_salary
        FROM ds_jobs_fs
        WHERE state IS NOT NULL
        GROUP BY state
        ORDER BY 2 DESC; 
    """)
 	
    # Carregar em um DataFrame
    df = pd.read_sql(analyze_query, engine)
    # Exibir o resultado
    display(df)

,state,vagas_por_estado,average_salary,median_salary
0,CA,155,120419.35,114000.00
1,VA,89,126769.66,115500.00
2,MA,62,121967.74,113500.00
3,NY,52,136432.69,120500.00
4,MD,40,112375.00,106000.00
5,IL,30,120883.33,114750.00
6,DC,26,139500.00,115500.00
7,TX,18,133888.89,114000.00
8,WA,16,134781.25,117250.00
9,OH,14,121714.29,106500.00


##### Faixa salarial por **setor**:

Esta consulta busca extrair dados sobre salários estimados para diversos setores. Os resultados são agrupados pelo campo `sector` e organizados em ordem dos maiores sálários.

In [197]:
with engine.connect() as conn:
    analyze_query = text("""
        SELECT 
            sector,
            MIN(salary_estimate_lower),
            MAX(salary_estimate_higher)
        FROM ds_jobs_fs
        WHERE sector IS NOT NULL
        GROUP BY sector
        ORDER BY 3 DESC; 
    """)
 	
    # Carregar em um DataFrame
    df = pd.read_sql(analyze_query, engine)
    # Exibir o resultado
    display(df)

,sector,min,max
0,Information Technology,31000,331000
1,Business Services,31000,331000
2,Aerospace & Defense,31000,331000
3,Consumer Services,105000,331000
4,Media,90000,331000
5,Biotech & Pharmaceuticals,31000,331000
6,Travel & Tourism,66000,225000
7,Insurance,31000,225000
8,Health Care,56000,225000
9,Government,56000,225000


Agora examinarei a distribuição de vagas e a variação dos salários médios estimados por setor. Setores com alto desvio padrão podem incluir cargos de níveis variados ou especializações únicas.

In [199]:
with engine.connect() as conn:
    analyze_query = text("""
        SELECT 
            sector,
            COUNT(sector) AS vagas_por_setor,
            STDDEV_SAMP(salary_estimate_avg) as salary_stddev
        FROM ds_jobs_fs
        WHERE sector IS NOT NULL
        GROUP BY sector
        HAVING COUNT(sector) != 1
        ORDER BY 3 DESC; 
    """)
 	
    # Carregar em um DataFrame
    df = pd.read_sql(analyze_query, engine)
    # Exibir o resultado
    display(df)

,sector,vagas_por_setor,salary_stddev
0,Consumer Services,2,95812.97
1,Media,5,72973.11
2,Travel & Tourism,3,48686.07
3,Business Services,120,46595.70
4,Aerospace & Defense,46,46590.59
5,"Construction, Repair & Maintenance",2,44194.17
6,Biotech & Pharmaceuticals,66,43408.64
7,Retail,7,35006.80
8,Manufacturing,23,34583.85
9,Information Technology,187,33393.04


Por fim, agrego o número de vagas e calculo a média e a mediana dos salários estimados. Ela filtra registros sem setor, agrupa os dados por setor e em ordem decrescente pelo número de vagas.

Setores com poucas vagas podem ter estatísticas menos represetativas.

In [201]:
with engine.connect() as conn:
    analyze_query = text("""
        SELECT 
            sector,
            COUNT(sector) AS vagas_por_setor,
            AVG(salary_estimate_avg) AS average_salary,
            PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY salary_estimate_avg) AS median_salary
        FROM ds_jobs_fs
        WHERE sector IS NOT NULL
        GROUP BY sector
        ORDER BY 2 DESC; 
    """)
 	
    # Carregar em um DataFrame
    df = pd.read_sql(analyze_query, engine)
    # Exibir o resultado
    display(df)

,sector,vagas_por_setor,average_salary,median_salary
0,Information Technology,187,119013.37,113000.00
1,Business Services,120,130079.17,114000.00
2,Biotech & Pharmaceuticals,66,122871.21,106500.00
3,Aerospace & Defense,46,132695.65,123500.00
4,Insurance,32,111281.25,106000.00
5,Finance,32,116000.00,113000.00
6,Manufacturing,23,123239.13,115500.00
7,Health Care,21,119761.90,114000.00
8,Government,17,134470.59,136500.00
9,"Oil, Gas, Energy & Utilities",10,101300.00,101250.00


#### Resumo:

**Análise por Cargo:**

A análise salarial por cargo revela que posições como **'Other Scientist', 'Data Scientist' e 'ML/AI Engineer/Scientist'** apresentam as médias e medianas salariais mais elevadas (geralmente acima de 120k para média e 107k-136k para mediana). No entanto, essas mesmas categorias exibem a **maior variabilidade salarial** (desvio padrão > 40k), indicando uma ampla dispersão nos salários devido a fatores como senioridade, localização e especialização. **'Data Scientist'** é, de longe, o cargo com maior volume de vagas (443). Cargos como **'Data Engineer'** mostram salários competitivos (média ~114k) com variabilidade notavelmente menor (stddev ~23k), sugerindo maior consistência salarial. Cargos como **'Analytics Manager' e 'Research Scientist'** aparecem com menor volume e salários médios/medianos mais baixos neste dataset.

**Análise por Estado:**

> - Regionalmente, a maior concentração de vagas encontra-se nos estados da **Califórnia (CA, 155 vagas), Virgínia (VA, 89), Massachusetts (MA, 62) e Nova York (NY, 52)**. 
> - Em termos de remuneração, estados como **Carolina do Norte (NC), Distrito de Columbia (DC), Nova York (NY), Texas (TX) e Washington (WA)** apresentam médias salariais elevadas (acima de 130k), o que pode refletir fatores como o alto custo de vida e a concentração de grandes empresas nessas regiões. 
> - A categoria **'Remote'** também aparece, com salários médios competitivos (127k).

Contudo, a **variabilidade salarial dentro dos estados é significativa**, como evidenciado pelo alto desvio padrão em **DC (62k), WA (56k) e NY (48k)**, enquanto estados como **CA (32k) e IL (29k)**, apesar do alto volume de vagas, mostram menor dispersão relativa. É crucial interpretar com cautela os dados de estados com poucas vagas (ex: OK, RI, IA), pois suas estatísticas podem ser influenciadas por poucos anúncios. 

**Análise por Setor:**

> - **Tecnologia da Informação (187 vagas), Serviços de Negócios (120) e Biotecnologia/Farmacêutica (66)** são os setores com maior número de anúncios.
> - Setores como **Mídia, Serviços ao Consumidor, Varejo, Aeroespacial/Defesa e Governo** exibem as maiores médias e medianas salariais (muitas vezes acima de 130k).

No entanto, alguns desses setores mais bem pagos (ex: **Serviços ao Consumidor, Mídia**) têm poucas vagas e/ou **alta variabilidade salarial** (stddev > 70k), tornando as médias menos representativas. Setores volumosos como **Tecnologia da Informação (stddev 33k), Finanças (33k) e Seguros (28k)** oferecem salários competitivos (média > 110k) com variabilidade mais moderada, sugerindo maior previsibilidade salarial em comparação a setores mais nichados ou diversos.

<hr style="border: none; border-top: 1px dashed #ccc;">

#### Empresas que oferecem mais vagas:

O objetivo desta consulta é identificar as combinações de empresa, estado, sede e setor que possuem um volume significativo (mais de 4) de vagas de emprego na tabela `ds_jobs_fs`. A consulta filtra dados incompletos e ordena os resultados para destacar as ocorrências mais frequentes.

In [206]:
with engine.connect() as conn:
    analyze_query = text("""
        SELECT
            company,
            state,
            headquarters,
            sector,
            COUNT(*) as numero_vagas,
            AVG(salary_estimate_avg) as salary_estimate_avg
        FROM ds_jobs_fs
        WHERE company IS NOT NULL
          AND sector IS NOT NULL
          AND headquarters IS NOT NULL
          AND state IS NOT NULL
        GROUP BY company, state, headquarters, sector
        HAVING COUNT(*) > 4
        ORDER BY numero_vagas DESC;
    """)
    
    # Carregar em um DataFrame
    df = pd.read_sql(analyze_query, engine)
    # Exibir o resultado
    display(df)

,company,state,headquarters,sector,numero_vagas,salary_estimate_avg
0,Maxar Technologies,VA,"Westminster, CO",Aerospace & Defense,12,150958.33
1,Klaviyo,MA,"Boston, MA",Information Technology,8,133312.50
2,Phoenix Operations Group,MD,"Woodbine, MD",Information Technology,7,99857.14
3,AstraZeneca,MD,"Cambridge, United Kingdom",Biotech & Pharmaceuticals,7,103214.29
4,Novetta,VA,"Mc Lean, VA",Information Technology,6,129916.67
5,Autodesk,CA,"San Rafael, CA",Information Technology,6,119416.67
6,Tempus Labs,CA,"Chicago, IL",Biotech & Pharmaceuticals,6,123083.33
7,Novartis,MA,"Basel, Switzerland",Biotech & Pharmaceuticals,5,111000.00
8,MassMutual,MA,"Springfield, MA",Insurance,5,95800.00
9,Tempus Labs,IL,"Chicago, IL",Biotech & Pharmaceuticals,5,116500.00


Esta consulta conta a frequência de cada cargo (`job_title_simplified`) por empresa. Ela foca a análise apenas nas empresas acima.

In [208]:
with engine.connect() as conn:
    analyze_query = text("""
        SELECT 
           job_title_simplified AS cargo,
           company, 
           COUNT(job_title_simplified)
        FROM ds_jobs_fs
        WHERE company IN (SELECT company
                   FROM ds_jobs_fs
                   WHERE company IS NOT NULL
                     AND sector IS NOT NULL
                     AND headquarters IS NOT NULL
                     AND state IS NOT NULL
                   GROUP BY company, state, headquarters, sector
                   HAVING COUNT(*) > 4)
        GROUP BY company, job_title_simplified
        ORDER BY job_title_simplified, COUNT(job_title_simplified) DESC
    """)
    
    # Carregar em um DataFrame
    df = pd.read_sql(analyze_query, engine)
    # Exibir o resultado
    display(df)

,cargo,company,count
0,Data Analyst,Autodesk,2
1,Data Analyst,Novetta,2
2,Data Engineer,Maxar Technologies,1
3,Data Scientist,Maxar Technologies,11
4,Data Scientist,AstraZeneca,10
5,Data Scientist,Klaviyo,7
6,Data Scientist,Phoenix Operations Group,7
7,Data Scientist,Autodesk,5
8,Data Scientist,Novetta,4
9,Data Scientist,Novartis,3


#### Resumo:

A análise identificou empresas com mais de quatro vagas, destacando:  
- **Maxar Technologies** como líder, com 12 oportunidades em Virginia e a maior média salarial de **150k** no setor de *Aerospace & Defense*.  
- **Klaviyo** segue com 8 vagas em Massachusetts, oferecendo **133k** em média no setor de *Information Technology*.  
- **Phoenix Operations Group** e **AstraZeneca**, ambas com 7 vagas em Maryland, mas com médias salariais distintas de **99k** e **103k**, respectivamente.
- **Novetta** e **Autodesk**, com 6 vagas cada, também se destacam, com médias salariais competitivas de **129k** e **119k**.  

A investigação sobre a variedade de cargos revelou: 
- **AstraZeneca** e **Maxar Technologies** concentram suas contratações em Data Scientists, com 10 e 11 vagas, respectivamente.
- **Tempus Labs** foca em ML/AI Engineers, com 9 vagas.
- **Klaviyo** e **Autodesk** também priorizam Data Scientists, com 7 e 5 vagas, respectivamente.
- **Novetta** apresentam uma  uma demanda diversificada por profissionais de dados em Data Analysts e Data Scientists.
- **Phoenix Operations Group** possui exclusivamente 7 vagas para Data Scientists.

<hr style="border: none; border-top: 1px dashed #ccc;">

#### Cruzamento: Salários por Estado/Setor vs. Empresas Líderes em Contratação:

1.  **Concentração Geográfica e Setorial:**
    *   As empresas que mais contratam (Maxar, Klaviyo, Phoenix, AstraZeneca, Novetta, Autodesk, Tempus, Novartis, MassMutual) estão predominantemente localizadas nos estados com **maior volume geral de vagas**: **VA (Maxar, Novetta), MA (Klaviyo, Novartis, MassMutual), MD (Phoenix, AstraZeneca), CA (Autodesk, Tempus), e IL (Tempus)**. Isso confirma que a atividade de contratação significativa dessas empresas está alinhada com os principais hubs geográficos identificados na análise de salários por estado.

    *   Os setores dessas empresas (**Aerospace & Defense, Information Technology, Biotech & Pharmaceuticals, Insurance**) também estão entre os setores com **maior volume de vagas** na análise geral.

<blamk>

2.  **Salários das Empresas vs. Salários Gerais do Estado/Setor:**
    *   **Maxar Technologies (VA, Aero/Defense – 151k avg):** Opera em VA (2º maior volume, média 127k, mediana 115.5k) e no setor Aero/Defense (4º maior volume, média 133k, mediana 123.5k). A média salarial de Maxar (151k) é **significativamente superior** à média geral tanto de VA quanto do setor Aero/Defense, sugerindo que a empresa paga um prêmio considerável, possivelmente por posições mais seniores ou especializadas.
      
    *   **Klaviyo (MA, IT – 133k avg):** Opera em MA (3º maior volume, média 122k, mediana 113.5k) e no setor de TI (1º maior volume, média 119k, mediana 113k). A média de Klaviyo (133k) está **acima** da média de MA e do setor de TI, posicionando-a como uma empresa que paga bem dentro do seu nicho em MA.
  
    *   **Phoenix Ops (MD, IT – 100k avg) & AstraZeneca (MD, Biotech – 103k avg):** Operam em MD (5º maior volume, média 112k, mediana 106k). Seus setores são TI (média 119k) e Biotech (média 123k). As médias salariais de ambas as empresas (100k–103k) estão **abaixo** da média geral de MD e também abaixo da média geral de seus respectivos setores. Isso pode indicar que as vagas específicas em MD dessas empresas são de nível mais júnior, ou que MD, apesar de ter volume, tem um nível salarial geral ligeiramente menor para esses setores comparado a outros hubs.
  
    *   **Novetta (VA, IT – 130k avg):** Opera em VA (média 127k) e TI (média 119k). Sua média salarial (130k) está **acima** da média de VA e do setor de TI, similar à Klaviyo.
  
    *   **Autodesk (CA, IT – 119k avg):** Opera em CA (1º maior volume, média 120k, mediana 114k) e TI (média 119k). A média de Autodesk (119k) está **praticamente alinhada** com a média geral de CA e do setor de TI.
    *   **Tempus Labs (CA/IL, Biotech – 123k CA avg, 117k IL avg):** Opera em CA (média 120k) e IL (6º maior volume, média 121k, mediana 115k), no setor Biotech (média 123k). A média em CA (123k) está alinhada com a média do setor e ligeiramente acima da média de CA. A média em IL (117k) está um pouco abaixo da média de IL e da média do setor Biotech.
    *   **Novartis (MA, Biotech – 111k avg):** Opera em MA (média 122k) e Biotech (média 123k). Sua média salarial (111k) está **abaixo** da média de MA e do setor Biotech.
    *   **MassMutual (MA, Insurance – 96k avg):** Opera em MA (média 122k) e Insurance (5º maior volume, média 111k, mediana 106k). Sua média salarial (96k) está **significativamente abaixo** da média de MA e do setor de Seguros. Isso pode ser influenciado pela localização específica (Springfield vs. Boston) ou pelos tipos de cargos (inclui 3 "Other").

**Conclusão:**

*   **Empresas Líderes em Hubs Principais:** As empresas com maior volume de contratação (nessas combinações específicas) estão de fato localizadas nos estados e atuam nos setores que dominam o mercado de vagas de dados neste dataset.

*   **Variação Salarial entre Empresas Líderes:** Mesmo entre as empresas que mais contratam, existe uma variação considerável na remuneração oferecida. Algumas (Maxar, Klaviyo, Novetta) parecem pagar um prêmio em relação às médias estaduais/setoriais, enquanto outras (Phoenix, AstraZeneca, Novartis, MassMutual) oferecem salários médios abaixo dessas médias gerais, e algumas (Autodesk, Tempus) estão mais alinhadas.
*   **Influência do Estado/Setor é Contextual:** Saber a média salarial de um estado ou setor fornece um bom contexto, mas não determina o salário de uma empresa específica. Empresas podem pagar acima ou abaixo dessa média dependendo de sua própria estratégia de remuneração, do nível das vagas, da localização exata dentro do estado e da competitividade do nicho específico.
*   **Foco em Data Scientist:** A predominância de "Data Scientist" como cargo principal nessas empresas líderes, combinada com a alta variabilidade salarial vista para esse cargo na análise geral, sugere que o título abrange uma gama ampla de senioridade e especialização, refletindo-se nas diferentes médias salariais oferecidas pelas empresas.

---
### Conclusão:

**1. O que foi observado sobre as relações entre as variáveis?**

*   **Habilidades vs. Cargo:** Existe uma forte relação entre o cargo (`job_title_simplified`) e as habilidades mais requisitadas. Por exemplo:
    *   Data Scientists: Forte demanda por Python, Machine Learning e SQL.
    *   Data Analysts: Forte demanda por SQL, Python e Tableau.
    *   Data Engineers: Forte demanda por Python, Cloud e SQL.
    *   ML/AI Engineers/Scientists: Quase universal a demanda por Machine Learning e Python, com alta demanda por Cloud.
    *   Analytics Manager: Demanda por SQL e Tableau (100% neste dataset).

<blank>

*   **Habilidades vs. Setor:** A demanda por habilidades específicas varia significativamente por setor.
    *   Tecnologia da Informação: Alta demanda por Python, ML, Cloud e SQL.
    *   Serviços de Negócios: Semelhante a TI, mas com maior ênfase em Tableau.
    *   Finanças e Seguros: Maior demanda relativa por SQL, Tableau e Excel.

<blank>

*   **Habilidades vs. Salário (Mediana):** A relação mais notável foi:
    *   Presença de **Power BI** associada a uma mediana salarial *ligeiramente superior*.
    *   Presença de **Excel** associada a uma mediana salarial *ligeiramente inferior*.
    *   Presença das habilidades mais comuns (Python, SQL, ML, Cloud, Tableau) *não mostrou diferença* na mediana salarial geral, sugerindo que são habilidades base ou que outros fatores (senioridade, localização, combinações) são mais determinantes para a remuneração mediana neste dataset.

<blank>

*   **Salário vs. Cargo:** Cargos como 'Other Scientist', 'Data Scientist', e 'ML/AI Engineer/Scientist' tendem a ter médias/medianas salariais mais altas, mas também a maior variabilidade (desvio padrão). 'Data Engineer' tem salário competitivo com menor variabilidade.

<blank>

*   **Salário vs. Estado:** Estados como CA, VA, MA, NY concentram vagas. Estados como NC, DC, NY, TX, WA mostram médias salariais mais altas, mas também alta variabilidade em alguns casos (DC, WA, NY). A localização 'Remote' também é competitiva.

<blank>

*   **Salário vs. Setor:** Setores como Mídia, Serviços ao Consumidor, Varejo, Aeroespacial/Defesa e Governo mostraram as médias/medianas mais altas, mas frequentemente com baixo volume de vagas e/ou alta variabilidade. Setores com mais volume (TI, Finanças, Seguros, Biotech) oferecem salários competitivos com variabilidade mais moderada.

<blank>

*   **Empresas vs. Localização/Setor/Salário:** As empresas que mais contratam estão nos principais hubs geográficos e setores. No entanto, seus níveis salariais variam: algumas pagam acima da média local/setorial (ex: Maxar, Klaviyo), outras abaixo (ex: Phoenix, AstraZeneca, MassMutual), e algumas alinhadas (ex: Autodesk).

<blank>

**2. O que foi observado sobre as distribuições nos dados?**

*   **Distribuição de Cargos (`job_title_simplified`):** A distribuição é desbalanceada, com 'Data Scientist' sendo a categoria mais frequente (443 vagas), seguida por outras categorias com volumes bem menores (Analyst, Engineer, ML/AI com 46-49 vagas cada). A categoria 'Other' foi reduzida a 12 registros após a simplificação.

<blank>
    
*   **Distribuição de Habilidades:** Python (477), Machine Learning (409) e SQL (323) são as habilidades mais mencionadas no geral. Cloud (230) e Tableau (122) também são significativas. Excel (78) e Power BI (48) são menos frequentes neste conjunto de dados.

<blank>

*   **Distribuição Geográfica (Estados):** A distribuição de vagas por estado é concentrada, com CA (155), VA (89), MA (62) e NY (52) liderando. Muitos outros estados têm um número muito baixo de vagas.

<blank>

*   **Distribuição Setorial:** A distribuição por setor também é concentrada, com TI (187), Serviços de Negócios (120) e Biotecnologia/Farmacêutica (66) tendo o maior número de vagas. Muitos outros setores têm poucas vagas representadas.

<blank>

*   **Distribuição Salarial (`salary_estimate_avg`):**
    *   A mediana geral parece estar em torno de 114k (observado na análise de habilidades vs. salário).
    *   Há uma **alta variabilidade (desvio padrão)** nos salários dentro de certas categorias de cargos (Data Scientist, ML/AI, Other Scientist), estados (DC, WA, NY) e setores (Mídia, Serviços ao Consumidor), indicando uma ampla gama de salários possivelmente devido a senioridade, especialização, custo de vida, etc.
    *   Outras categorias (ex: Data Engineer como cargo, CA/IL como estados, TI/Finanças/Seguros como setores) mostram menor variabilidade relativa.

<blank>

**3. Que transformações foram feitas nos dados? Por que essas decisões foram tomadas?**

As seguintes transformações foram realizadas:

1.  **Criação de Flags de Habilidades (`tem_sql`, `tem_python`, etc.):**
    *   **Transformação:** Adição de colunas binárias (0 ou 1) indicando a presença de palavras-chave de habilidades na coluna `job_description` usando `LOWER()` e expressões regulares (`~ '\msql\M'`).
    *   **Porquê:** Para facilitar e otimizar a análise quantitativa das habilidades: contagem de vagas por skill, identificação de combinações de skills, e comparação de salários médios/medianos entre vagas que exigem ou não uma habilidade específica.

<blank>

2.  **Criação de `salary_estimate_avg`:**
    *   **Transformação:** Adição de uma coluna numérica calculada como a média entre `salary_estimate_lower` e `salary_estimate_higher`.
    *   **Porquê:** Para obter um valor numérico único representando o salário estimado, simplificando cálculos estatísticos (média, mediana, desvio padrão) e análises comparativas (por localização, setor, habilidades).

<blank>

3.  **Criação de `state`:**
    *   **Transformação:** Adição de uma coluna extraindo a sigla do estado (últimos 2 caracteres) da coluna `location` (formato "Cidade, ST"). Casos especiais como 'Remote' e locais com apenas 2 caracteres foram tratados.
    *   **Porquê:** Para permitir a agregação e análise de dados (vagas, salários) por estado/região de forma mais fácil.

<blank>

4.  **Criação de `job_title_simplified`:**
    *   **Transformação:** Adição de uma coluna que agrupa títulos de cargos semelhantes em categorias mais amplas usando `CASE` e `ILIKE`.
    *   **Porquê:** Para reduzir a granularidade dos títulos de cargos, permitindo análises mais significativas por *tipo* de função (ex: comparar todos os Data Analysts) em vez de variações específicas (Senior, II, Lead). Facilita a comparação de salários e habilidades para funções equivalentes.

<blank>

5.  **Criação da Tabela `ds_jobs_fs` e Seleção de Features:**
    *   **Transformação:** Criação de uma nova tabela (`ds_jobs_fs`) como cópia de `ds_jobs`, e posterior remoção das colunas `id`, `competitors`, `job_description`, `salary_estimate`, `founded`.
    *   **Porquê:** Para criar um conjunto de dados mais enxuto (`Feature Selection`), mantendo apenas as variáveis consideradas mais relevantes ou úteis para as análises subsequentes, melhorando o foco, a interpretabilidade e a eficiência. `job_description` e `salary_estimate` foram substituídas pelas features derivadas (flags, avg salary). `id`, `competitors`, `founded` foram consideradas menos relevantes para as perguntas de análise salarial e de habilidades.

<blank>

6.  **Descarte da Feature `experience_level` (Decisão de *Não* Transformar):**
    *   **Transformação (Evitada):** Não foi criada uma feature para nível de experiência.
    *   **Porquê:** Devido à complexidade de extrair essa informação de forma confiável do texto da descrição da vaga usando apenas SQL padrão, que é o objetivo do projeto.

<blank>

**4. Há considerações éticas nesta etapa?**

Sim, existem considerações éticas importantes nesta etapa de análise e transformação:

*   **Viés nos Dados de Origem:** O conjunto de dados original (anúncios do Glassdoor) pode conter vieses inerentes. As vagas podem não representar todo o mercado, podendo haver sobrerrepresentação de certos setores, locais, tipos de empresa ou níveis de senioridade. As descrições e salários publicados podem refletir vieses (conscientes ou inconscientes) das empresas anunciantes.

<blank>

*   **Generalização e Representatividade:** Os insights são baseados *neste dataset específico*. Generalizar as conclusões (ex: "Excel sempre paga menos", "Power BI sempre paga mais") para todo o mercado de trabalho de ciência de dados seria inadequado e potencialmente prejudicial se usado para aconselhamento de carreira ou decisões de contratação sem contexto adicional.

<blank>

Em resumo, a etapa de análise e transformação foi focada em extrair insights dos dados disponíveis, mas a interpretação e comunicação desses insights devem ser feitas com responsabilidade, reconhecendo as limitações dos dados e da metodologia.